## Identifying and resolving conflicts in a batch of VI files

ELGs - part 2 is sets 5 to 10

There are 849 visual inspections of a total of 300 unique objects

80 conflicts

In [2]:
save_filename='truth_table_ELGs_sets5_10.txt'

In [3]:
80/300

0.26666666666666666

In [4]:
import os, sys, glob
import numpy as np

from astropy.io import fits
from astropy.table import Table, join, vstack
from astropy.io import fits

import pandas as pd
import fnmatch

import desispec.io

#import desispec
# library location will change ..
sys.path.append("/global/homes/r/rtojeiro/prospect/prospect/py")
from prospect import utils_specviewer,plotframes
import matplotlib.pyplot as plt 

In [5]:
tiledir   = '/global/cfs/cdirs/desi/spectro/redux/daily/tiles/'
tiles = ['67230']
nights = ['20200315']
petals = ['0','1', '2', '3', '4', '5', '6' ,'7', '8', '9']
obs_db = utils_specviewer.make_targetdict(tiledir, petals=petals, tiles=tiles) # tiles = optional 

In [6]:
pd.set_option('display.max_rows', 20)

In [7]:
#set to directory with all the VI files to merge
VI_dir = os.environ['HOME']+'/SV/VI_files/SV0/ELG/'

In [8]:
#read a set of files 
all_files = os.listdir(VI_dir)
vi_files=[]

for i in range(5,11):
    pattern = "desi*_"+str(i)+"_*.csv"
    print(pattern)
    for entry in all_files:
        if fnmatch.fnmatch(entry, pattern):
            vi_files.append(entry)

vi_files

desi*_5_*.csv
desi*_6_*.csv
desi*_7_*.csv
desi*_8_*.csv
desi*_9_*.csv
desi*_10_*.csv


['desi-vi_SV0_ELG_tile67230_night20200315_5_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_5_CBA.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_5_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_6_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_6_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_6_CBA.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_7_CBA.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_7_ACE.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_7_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_8_BJW.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_8_VRK.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_9_SEM.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_9_CGS.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_9_AJG.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_10_SEM.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_10_ASC.csv',
 'desi-vi_SV0_ELG_tile67230_night20200315_10_CGS.csv']

In [9]:
vi = pd.read_csv(VI_dir + vi_files[0], delimiter = " , ", engine='python')

for i in range(1,len(vi_files)):
    print(vi_files[i])
    vi2 = pd.read_csv(VI_dir + vi_files[i], delimiter = " , ", engine='python')
    vi = vi.append(vi2, ignore_index=True)
    
#make groups of visual inspections, grouped by unique objects, and state number of single and multiple VIs
vi_gp = vi.groupby(['TargetID'])
print('There are ' + str(len(vi)) + ' visual inspections of a total of ' + str(len(vi_gp)) + ' unique objects')
print('That gives ' + str(len(vi)/len(vi_gp)) + ' inspections per object.')

desi-vi_SV0_ELG_tile67230_night20200315_5_CBA.csv
desi-vi_SV0_ELG_tile67230_night20200315_5_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_6_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_6_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_6_CBA.csv
desi-vi_SV0_ELG_tile67230_night20200315_7_CBA.csv
desi-vi_SV0_ELG_tile67230_night20200315_7_ACE.csv
desi-vi_SV0_ELG_tile67230_night20200315_7_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_8_BJW.csv
desi-vi_SV0_ELG_tile67230_night20200315_8_VRK.csv
desi-vi_SV0_ELG_tile67230_night20200315_9_SEM.csv
desi-vi_SV0_ELG_tile67230_night20200315_9_CGS.csv
desi-vi_SV0_ELG_tile67230_night20200315_9_AJG.csv
desi-vi_SV0_ELG_tile67230_night20200315_10_SEM.csv
desi-vi_SV0_ELG_tile67230_night20200315_10_ASC.csv
desi-vi_SV0_ELG_tile67230_night20200315_10_CGS.csv
There are 849 visual inspections of a total of 300 unique objects
That gives 2.83 inspections per object.


In [10]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment'],
      dtype='object')

### Merge with zbest files
Add: fiberID, delta_chi2, flux information,.. anything else?


In [11]:

#read in fibermap info, loop over the files for all the petals
tf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
tspec = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[0])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
for i in range(1,len(petals)):
    tn = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='ZBEST')
    tnf = Table.read(tiledir+'/'+tiles[0] + '/'+nights[0]+'/zbest-'+str(petals[i])+'-'+str(tiles[0])+'-'+nights[0]+'.fits',hdu='FIBERMAP')
    tspec = vstack([tspec,tn])
    tf = vstack([tf,tnf])

tspec_df = tspec['TARGETID','DELTACHI2' ].to_pandas()
tf_df = tf['TARGETID','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].to_pandas()

tf_df = tf_df.rename(columns={"TARGETID": "TargetID"})
tspec_df = tspec_df.rename(columns={"TARGETID": "TargetID"})

vi = vi.merge(tf_df, how='left', on='TargetID')
vi = vi.merge(tspec_df, how='left', on='TargetID')

In [12]:
print(vi.keys())
print(len(vi))

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2'],
      dtype='object')
849


### Adding a bunch of useful columns

In [13]:
#make new column with best redshift estimate for each VI - take VI redshift if available, else take Redrock redshift. 
#I am always assuming that the VI redshift, if provided, trumps over the Redrock redshift. 
vi['best redshift'] = vi['VI z']
vi.loc[vi['best redshift']=='--', 'best redshift'] = vi.loc[vi['best redshift']=='--', 'Redrock z']
vi.loc[vi['best redshift']=='>5', 'best redshift'] = 5 #catch someone who wrote >5 in VI z field
vi['best redshift'] = vi['best redshift'].astype(float)

In [14]:
#make new column with best spectype estimate for each VI - take VI spectype if available, else take Redrock spectype 
#I am always assuming that the VI redshift, if provided, trumps over the Redrock redshift. 
vi['best spectype'] = vi['VI spectype']
vi.loc[vi['best spectype']=='--', 'best spectype'] = vi.loc[vi['best spectype']=='--', 'Redrock spectype']

In [15]:
#add new columns, holding the mean of the flags and the maximum difference in flag classification
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_diff'] = vi.groupby('TargetID')['VI class'].transform(lambda x: ( x.max()-x.min()) )

In [16]:
#add new column, with the mean redshift from all values of 'best redshift'
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')
vi['dz'] = vi.groupby('TargetID')['best redshift'].transform(lambda x: ( (x.max() - x.min()) / (1+x.min()) ))

In [17]:
#add new column, with all comments concatenated
vi['all VI comments'] = vi.groupby('TargetID')['VI comment'].transform(lambda x: '|'.join(x))

In [18]:
#add new column, with the number of VI inspections for each object
vi['N_VI'] = vi.groupby('TargetID')['TargetID'].transform('count')

In [19]:
#add new column to hold comments from merger if needed
vi['merger comment'] = 'none'

Get a table that holds only the objects that have been inspected more than once, and for which the individual VI classifications differ by 2 or more, or delta z / (1 + z) > 0.0033, or there is disagreement in best spectype (these are the conflicts to resolve)

In [20]:
vi_gp = vi.groupby(['TargetID'])
vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
                       & (len(x) >= 2)) #x is a group by TargetID

Get the target IDs of the problematic objects and display in table form for a quick summary:

In [21]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185741969491373 35185741969493081 35185741969493978 35185741969494246
 35185741973685286 35185741973688584 35185741973688595 35185741977880595
 35185741977880645 35185741977881344 35185741977882025 35185741977882226
 35185741977882979 35185747984122831 35185747988316811 35185747988318730
 35185747988321049 35185747988321076 35185747992511204 35185747992511879
 35185747992512523 35185747992514594 35185747996705618 35185747996705619
 35185747996707619 35185747996707757 35185747996708490 35185747996708934
 35185747996709244 35185747996710287 35185748000899521 35185748000900278
 35185748000901009 35185748000901678 35185748000903455 35185748000903650
 35185748005094056 35185748005095141 35185748005095972 35185748005096260
 35185748005097030 35185748005097490 35185748009288493 35185748009288692
 35185748009288821 35185748009289720 35185748009290088 35185748009290318
 35185748009290333 35185748009292017 35185748013486472 35185748017676546
 35185748017677288 35

In [22]:
unique_targets[1]

35185741969493081

## This is where I resolve things manually - with care!!
### I think it's better to keep it in a notebook, as typos can be backtracked rather than a single manual edit of a text file

We edit either 'VI class', or 'best redshift' to resolve conflict. At the end, we look for conflicts again and we should find none.


In [23]:
#function to display the conflict in table format and open a prospect window
def display_conflict(conflict_id, show_spectra=None):
    
    #first, remind myself of the problem:
    display(vi[vi.TargetID==unique_targets[conflict_id]][['TargetID', 
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment']])

    spectra, zcat= utils_specviewer.load_spectra_zcat_from_targets([unique_targets[conflict_id]], tiledir, obs_db)
    # VI interface in notebook
    if show_spectra:
        plotframes.plotspectra(spectra, zcatalog=zcat, title='Target_select', notebook=True, mask_type='CMX_TARGET',with_vi_widgets=False)

In [24]:
#first, keep a safe copy of the original dataframe
vi_safe = vi.copy()

We will inspect each conflict on a prospect window, and resolve each conflict in turn

In [25]:
#Keep track of the conflicts by conflict_id

#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
0,35185741969491373,GALAXY,0.5809,VRK,1,R,0.9699,--,dubious fit (doublet from noise + weak Hbeta ...,0.9699,GALAXY,2.0,2,0.710567,0.246062,dubious fit (doublet from noise + weak Hbeta ...,3,none
50,35185741969491373,GALAXY,0.5809,CBA,2,--,--,--,Possible [OII] & Balmer,0.5809,GALAXY,2.0,2,0.710567,0.246062,dubious fit (doublet from noise + weak Hbeta ...,3,none
100,35185741969491373,GALAXY,0.5809,ACE,3,--,--,--,--,0.5809,GALAXY,2.0,2,0.710567,0.246062,dubious fit (doublet from noise + weak Hbeta ...,3,none


In [26]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5809
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. continuum fit, but relies on blue calibration being right. low flag'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
0,35185741969491373,-1,0.33.0.dev3890,0,GALAXY,0.5809,VRK,2,R,0.9699,...,6.757586,0.5809,GALAXY,2.0,2,0.710567,0.246062,dubious fit (doublet from noise + weak Hbeta ...,3,"disagreement in flag and z. continuum fit, but..."
50,35185741969491373,-1,0.33.0.dev3890,0,GALAXY,0.5809,CBA,2,--,--,...,6.757586,0.5809,GALAXY,2.0,2,0.710567,0.246062,dubious fit (doublet from noise + weak Hbeta ...,3,"disagreement in flag and z. continuum fit, but..."
100,35185741969491373,-1,0.33.0.dev3890,0,GALAXY,0.5809,ACE,2,--,--,...,6.757586,0.5809,GALAXY,2.0,2,0.710567,0.246062,dubious fit (doublet from noise + weak Hbeta ...,3,"disagreement in flag and z. continuum fit, but..."


conflict ID =  0


In [27]:
#copy this text to a new cell to display the conflict
conflict_id=1
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
16,35185741969493081,GALAXY,0.8229,VRK,4,R,1.4485,--,doublet mistaken as OIII line at 9130AA + VI ...,1.4485,GALAXY,3.0,2,1.448533,0.000041,doublet mistaken as OIII line at 9130AA + VI ...,3,none
66,35185741969493081,GALAXY,0.8229,CBA,2,R,1.4485,--,Tentative [OII] at 9130A,1.4485,GALAXY,3.0,2,1.448533,0.000041,doublet mistaken as OIII line at 9130AA + VI ...,3,none
116,35185741969493081,GALAXY,0.8229,ACE,3,R,1.4486,GALAXY,OII fit as OIII with double peak that is not ...,1.4486,GALAXY,3.0,2,1.448533,0.000041,doublet mistaken as OIII line at 9130AA + VI ...,3,none


In [28]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but 3/3 inspectors agree on new redshift. secure z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
16,35185741969493081,-1,0.33.0.dev3890,0,GALAXY,0.8229,VRK,3,R,1.4485,...,47.458578,1.4485,GALAXY,3.0,2,1.448533,0.000041,doublet mistaken as OIII line at 9130AA + VI ...,3,"disagreement in flag, but 3/3 inspectors agree..."
66,35185741969493081,-1,0.33.0.dev3890,0,GALAXY,0.8229,CBA,3,R,1.4485,...,47.458578,1.4485,GALAXY,3.0,2,1.448533,0.000041,doublet mistaken as OIII line at 9130AA + VI ...,3,"disagreement in flag, but 3/3 inspectors agree..."
116,35185741969493081,-1,0.33.0.dev3890,0,GALAXY,0.8229,ACE,3,R,1.4486,...,47.458578,1.4486,GALAXY,3.0,2,1.448533,0.000041,doublet mistaken as OIII line at 9130AA + VI ...,3,"disagreement in flag, but 3/3 inspectors agree..."


conflict ID =  1


In [29]:
#copy this text to a new cell to display the conflict
conflict_id = 2
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
13,35185741969493978,GALAXY,0.3253,VRK,2,--,0.5608,--,all features (incduding doublet) are weak + VI...,0.5608,GALAXY,2.666667,2,0.4038,0.177696,all features (incduding doublet) are weak + VI...,3,none
63,35185741969493978,GALAXY,0.3253,CBA,4,--,--,--,"[OII]"","" Balmer lines (incl. Ha)"","" [OIII]",0.3253,GALAXY,2.666667,2,0.4038,0.177696,all features (incduding doublet) are weak + VI...,3,none
113,35185741969493978,GALAXY,0.3253,ACE,2,--,--,--,Noisy spectrum,0.3253,GALAXY,2.666667,2,0.4038,0.177696,all features (incduding doublet) are weak + VI...,3,none


In [30]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.3253
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. noisy, not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
13,35185741969493978,-1,0.33.0.dev3890,0,GALAXY,0.3253,VRK,2,--,0.5608,...,122.268946,0.3253,GALAXY,2.666667,2,0.4038,0.177696,all features (incduding doublet) are weak + VI...,3,"disagreement in z and flag. noisy, not secure."
63,35185741969493978,-1,0.33.0.dev3890,0,GALAXY,0.3253,CBA,2,--,--,...,122.268946,0.3253,GALAXY,2.666667,2,0.4038,0.177696,all features (incduding doublet) are weak + VI...,3,"disagreement in z and flag. noisy, not secure."
113,35185741969493978,-1,0.33.0.dev3890,0,GALAXY,0.3253,ACE,2,--,--,...,122.268946,0.3253,GALAXY,2.666667,2,0.4038,0.177696,all features (incduding doublet) are weak + VI...,3,"disagreement in z and flag. noisy, not secure."


conflict ID =  2


In [31]:
#copy this text to a new cell to display the conflict
conflict_id= 3
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
14,35185741969494246,GALAXY,1.6027,VRK,2,--,--,--,rather broad and partly resolved doublet in sk...,1.6027,GALAXY,1.0,2,1.6027,0.0,rather broad and partly resolved doublet in sk...,3,none
64,35185741969494246,GALAXY,1.6027,CBA,1,--,--,--,Hard to see [OII],1.6027,GALAXY,1.0,2,1.6027,0.0,rather broad and partly resolved doublet in sk...,3,none
114,35185741969494246,GALAXY,1.6027,ACE,0,--,--,--,No signal,1.6027,GALAXY,1.0,2,1.6027,0.0,rather broad and partly resolved doublet in sk...,3,none


In [32]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
14,35185741969494246,-1,0.33.0.dev3890,0,GALAXY,1.6027,VRK,1,--,--,...,21.655978,1.6027,GALAXY,1.0,2,1.6027,0.0,rather broad and partly resolved doublet in sk...,3,disagreement in flag 0 to 2. set to mean.
64,35185741969494246,-1,0.33.0.dev3890,0,GALAXY,1.6027,CBA,1,--,--,...,21.655978,1.6027,GALAXY,1.0,2,1.6027,0.0,rather broad and partly resolved doublet in sk...,3,disagreement in flag 0 to 2. set to mean.
114,35185741969494246,-1,0.33.0.dev3890,0,GALAXY,1.6027,ACE,1,--,--,...,21.655978,1.6027,GALAXY,1.0,2,1.6027,0.0,rather broad and partly resolved doublet in sk...,3,disagreement in flag 0 to 2. set to mean.


conflict ID =  3


In [33]:
#copy this text to a new cell to display the conflict
conflict_id=4
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35185741973685286,GALAXY,1.5308,VRK,1,S,--,--,possible partly resolved and rather strong do...,1.5308,GALAXY,2.0,2,1.5308,0.0,possible partly resolved and rather strong do...,3,none
70,35185741973685286,GALAXY,1.5308,CBA,2,--,--,--,Possible [OII],1.5308,GALAXY,2.0,2,1.5308,0.0,possible partly resolved and rather strong do...,3,none
120,35185741973685286,GALAXY,1.5308,ACE,3,--,--,--,--,1.5308,GALAXY,2.0,2,1.5308,0.0,possible partly resolved and rather strong do...,3,none


In [34]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
20,35185741973685286,-1,0.33.0.dev3890,0,GALAXY,1.5308,VRK,2,S,--,...,12.075732,1.5308,GALAXY,2.0,2,1.5308,0.0,possible partly resolved and rather strong do...,3,flag disagreement. set to mean.
70,35185741973685286,-1,0.33.0.dev3890,0,GALAXY,1.5308,CBA,2,--,--,...,12.075732,1.5308,GALAXY,2.0,2,1.5308,0.0,possible partly resolved and rather strong do...,3,flag disagreement. set to mean.
120,35185741973685286,-1,0.33.0.dev3890,0,GALAXY,1.5308,ACE,2,--,--,...,12.075732,1.5308,GALAXY,2.0,2,1.5308,0.0,possible partly resolved and rather strong do...,3,flag disagreement. set to mean.


conflict ID =  4


In [35]:
#copy this text to a new cell to display the conflict
conflict_id= 5
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
36,35185741973688584,GALAXY,0.5169,VRK,3,R,--,--,dubious fit (driven by one strong OIII line) ...,0.5169,GALAXY,2.666667,1,0.6905,0.343332,dubious fit (driven by one strong OIII line) ...,3,none
86,35185741973688584,GALAXY,0.5169,CBA,2,--,--,--,--,0.5169,GALAXY,2.666667,1,0.6905,0.343332,dubious fit (driven by one strong OIII line) ...,3,none
136,35185741973688584,GALAXY,0.5169,ACE,3,--,1.0377,GALAXY,OII misidentified as OIII,1.0377,GALAXY,2.666667,1,0.6905,0.343332,dubious fit (driven by one strong OIII line) ...,3,none


In [36]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0377
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with high confidence. one secure line, but doublet compromised by noise. noisy continuum. pick VI z with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
36,35185741973688584,-1,0.33.0.dev3890,0,GALAXY,0.5169,VRK,2,R,--,...,46.433842,1.0377,GALAXY,2.666667,1,0.6905,0.343332,dubious fit (driven by one strong OIII line) ...,3,disagreement in z with high confidence. one se...
86,35185741973688584,-1,0.33.0.dev3890,0,GALAXY,0.5169,CBA,2,--,--,...,46.433842,1.0377,GALAXY,2.666667,1,0.6905,0.343332,dubious fit (driven by one strong OIII line) ...,3,disagreement in z with high confidence. one se...
136,35185741973688584,-1,0.33.0.dev3890,0,GALAXY,0.5169,ACE,2,--,1.0377,...,46.433842,1.0377,GALAXY,2.666667,1,0.6905,0.343332,dubious fit (driven by one strong OIII line) ...,3,disagreement in z with high confidence. one se...


conflict ID =  5


In [37]:
#copy this text to a new cell to display the conflict
conflict_id = 6
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
37,35185741973688595,GALAXY,1.0248,VRK,1,R,1.0452,--,dubious fit (negative doublet + other lines u...,1.0452,GALAXY,1.0,0,1.0316,0.010075,dubious fit (negative doublet + other lines u...,3,none
87,35185741973688595,GALAXY,1.0248,CBA,1,--,--,--,--,1.0248,GALAXY,1.0,0,1.0316,0.010075,dubious fit (negative doublet + other lines u...,3,none
137,35185741973688595,GALAXY,1.0248,ACE,1,--,--,--,Featureless continuum,1.0248,GALAXY,1.0,0,1.0316,0.010075,dubious fit (negative doublet + other lines u...,3,none


In [38]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0452
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with low flag. set to one VI z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
37,35185741973688595,-1,0.33.0.dev3890,0,GALAXY,1.0248,VRK,1,R,1.0452,...,5.075543,1.0452,GALAXY,1.0,0,1.0316,0.010075,dubious fit (negative doublet + other lines u...,3,disagreement in z with low flag. set to one VI...
87,35185741973688595,-1,0.33.0.dev3890,0,GALAXY,1.0248,CBA,1,--,--,...,5.075543,1.0452,GALAXY,1.0,0,1.0316,0.010075,dubious fit (negative doublet + other lines u...,3,disagreement in z with low flag. set to one VI...
137,35185741973688595,-1,0.33.0.dev3890,0,GALAXY,1.0248,ACE,1,--,--,...,5.075543,1.0452,GALAXY,1.0,0,1.0316,0.010075,dubious fit (negative doublet + other lines u...,3,disagreement in z with low flag. set to one VI...


conflict ID =  6


In [39]:
#copy this text to a new cell to display the conflict
conflict_id=7
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
41,35185741977880595,GALAXY,0.6267,VRK,1,R,0.6389,--,dubious fit (negative doublet + other lines v...,0.6389,GALAXY,1.0,0,0.630767,0.0075,dubious fit (negative doublet + other lines v...,3,none
91,35185741977880595,GALAXY,0.6267,CBA,1,--,--,--,--,0.6267,GALAXY,1.0,0,0.630767,0.0075,dubious fit (negative doublet + other lines v...,3,none
141,35185741977880595,GALAXY,0.6267,ACE,1,--,--,--,Featureless blue continuum,0.6267,GALAXY,1.0,0,0.630767,0.0075,dubious fit (negative doublet + other lines v...,3,none


In [40]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6267
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with low flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
41,35185741977880595,-1,0.33.0.dev3890,0,GALAXY,0.6267,VRK,1,R,0.6389,...,0.662516,0.6267,GALAXY,1.0,0,0.630767,0.0075,dubious fit (negative doublet + other lines v...,3,disagreement in z with low flag.
91,35185741977880595,-1,0.33.0.dev3890,0,GALAXY,0.6267,CBA,1,--,--,...,0.662516,0.6267,GALAXY,1.0,0,0.630767,0.0075,dubious fit (negative doublet + other lines v...,3,disagreement in z with low flag.
141,35185741977880595,-1,0.33.0.dev3890,0,GALAXY,0.6267,ACE,1,--,--,...,0.662516,0.6267,GALAXY,1.0,0,0.630767,0.0075,dubious fit (negative doublet + other lines v...,3,disagreement in z with low flag.


conflict ID =  7


In [42]:
#copy this text to a new cell to display the conflict
conflict_id=8
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
42,35185741977880645,GALAXY,0.853,VRK,2,R,0.8536,--,dubious fit (multispikes in unsmoothed double...,0.8536,GALAXY,3.0,2,0.8532,0.000324,dubious fit (multispikes in unsmoothed double...,3,none
92,35185741977880645,GALAXY,0.853,CBA,3,--,--,--,Probable [OII] and [OIII],0.8530,GALAXY,3.0,2,0.8532,0.000324,dubious fit (multispikes in unsmoothed double...,3,none
142,35185741977880645,GALAXY,0.853,ACE,4,--,--,--,--,0.8530,GALAXY,3.0,2,0.8532,0.000324,dubious fit (multispikes in unsmoothed double...,3,none


In [43]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'conflict in flag. low SNR and unsecure line. flag as 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
42,35185741977880645,-1,0.33.0.dev3890,0,GALAXY,0.853,VRK,2,R,0.8536,...,30.866228,0.8536,GALAXY,3.0,2,0.8532,0.000324,dubious fit (multispikes in unsmoothed double...,3,conflict in flag. low SNR and unsecure line. f...
92,35185741977880645,-1,0.33.0.dev3890,0,GALAXY,0.853,CBA,2,--,--,...,30.866228,0.8530,GALAXY,3.0,2,0.8532,0.000324,dubious fit (multispikes in unsmoothed double...,3,conflict in flag. low SNR and unsecure line. f...
142,35185741977880645,-1,0.33.0.dev3890,0,GALAXY,0.853,ACE,2,--,--,...,30.866228,0.8530,GALAXY,3.0,2,0.8532,0.000324,dubious fit (multispikes in unsmoothed double...,3,conflict in flag. low SNR and unsecure line. f...


conflict ID =  8


In [44]:
#copy this text to a new cell to display the conflict
conflict_id = 9
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
43,35185741977881344,GALAXY,0.8796,VRK,0,RS,--,--,dubious fit (no doublet + fit driven by one u...,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,none
93,35185741977881344,GALAXY,0.8796,CBA,0,--,--,--,--,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,none
143,35185741977881344,GALAXY,0.8796,ACE,2,R,1.5252,GALAXY,OII on sky line,1.5252,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,none


In [45]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 0.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement 0 to 2. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
43,35185741977881344,-1,0.33.0.dev3890,0,GALAXY,0.8796,VRK,0.666667,RS,--,...,57.031874,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.
93,35185741977881344,-1,0.33.0.dev3890,0,GALAXY,0.8796,CBA,0.666667,--,--,...,57.031874,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.
143,35185741977881344,-1,0.33.0.dev3890,0,GALAXY,0.8796,ACE,0.666667,R,1.5252,...,57.031874,1.5252,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.


conflict ID =  9


In [48]:
#copy this text to a new cell to display the conflict
conflict_id= 10
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
48,35185741977882025,QSO,2.0134,VRK,2.0,--,--,--,not an ELG + not a clear QSO either,2.0134,QSO,0.666667,2,2.0134,0.0,not an ELG + not a clear QSO either|--| No si...,3,none
98,35185741977882025,QSO,2.0134,CBA,0.0,--,--,--,--,2.0134,QSO,0.666667,2,2.0134,0.0,not an ELG + not a clear QSO either|--| No si...,3,none
148,35185741977882025,QSO,2.0134,ACE,0.0,--,--,--,No signal,2.0134,QSO,0.666667,2,2.0134,0.0,not an ELG + not a clear QSO either|--| No si...,3,none


In [50]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 0.666667
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. set to mean. Issues with calibration'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
48,35185741977882025,-1,0.33.0.dev3890,0,QSO,2.0134,VRK,0.666667,--,--,...,4.728682,2.0134,QSO,0.666667,2,2.0134,0.0,not an ELG + not a clear QSO either|--| No si...,3,disagreement in flag 0 to 2. set to mean. Issu...
98,35185741977882025,-1,0.33.0.dev3890,0,QSO,2.0134,CBA,0.666667,--,--,...,4.728682,2.0134,QSO,0.666667,2,2.0134,0.0,not an ELG + not a clear QSO either|--| No si...,3,disagreement in flag 0 to 2. set to mean. Issu...
148,35185741977882025,-1,0.33.0.dev3890,0,QSO,2.0134,ACE,0.666667,--,--,...,4.728682,2.0134,QSO,0.666667,2,2.0134,0.0,not an ELG + not a clear QSO either|--| No si...,3,disagreement in flag 0 to 2. set to mean. Issu...


conflict ID =  10


In [53]:
#copy this text to a new cell to display the conflict
conflict_id=11
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
49,35185741977882226,GALAXY,1.2861,VRK,2.0,R,1.0694,--,dubiousfit (unconvincing doublet + other line...,1.0694,GALAXY,1.0,2,1.213867,0.104716,dubiousfit (unconvincing doublet + other line...,3,none
99,35185741977882226,GALAXY,1.2861,CBA,1.0,--,--,--,--,1.2861,GALAXY,1.0,2,1.213867,0.104716,dubiousfit (unconvincing doublet + other line...,3,none
149,35185741977882226,GALAXY,1.2861,ACE,0.0,--,--,--,No signal,1.2861,GALAXY,1.0,2,1.213867,0.104716,dubiousfit (unconvincing doublet + other line...,3,none


In [54]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0694
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. take one VI z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
49,35185741977882226,-1,0.33.0.dev3890,0,GALAXY,1.2861,VRK,1.0,R,1.0694,...,5.534907,1.0694,GALAXY,1.0,2,1.213867,0.104716,dubiousfit (unconvincing doublet + other line...,3,disagreement in flag 0 to 2. take one VI z wit...
99,35185741977882226,-1,0.33.0.dev3890,0,GALAXY,1.2861,CBA,1.0,--,--,...,5.534907,1.0694,GALAXY,1.0,2,1.213867,0.104716,dubiousfit (unconvincing doublet + other line...,3,disagreement in flag 0 to 2. take one VI z wit...
149,35185741977882226,-1,0.33.0.dev3890,0,GALAXY,1.2861,ACE,1.0,--,--,...,5.534907,1.0694,GALAXY,1.0,2,1.213867,0.104716,dubiousfit (unconvincing doublet + other line...,3,disagreement in flag 0 to 2. take one VI z wit...


conflict ID =  11


In [58]:
#copy this text to a new cell to display the conflict
conflict_id=12
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
197,35185741977882979,GALAXY,0.5128,VRK,2.0,R,0.7057,--,dubious fit (negative doublet and weak lines)...,0.7057,GALAXY,1.0,2,0.5771,0.127512,dubious fit (negative doublet and weak lines)...,3,none
246,35185741977882979,GALAXY,0.5128,ACE,0.0,--,--,--,No signal,0.5128,GALAXY,1.0,2,0.5771,0.127512,dubious fit (negative doublet and weak lines)...,3,none
296,35185741977882979,GALAXY,0.5128,CBA,1.0,--,--,--,--,0.5128,GALAXY,1.0,2,0.5771,0.127512,dubious fit (negative doublet and weak lines)...,3,none


In [59]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5128
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. took one VI redshift with flag 1. bad reduction'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
197,35185741977882979,-1,0.33.0.dev3890,0,GALAXY,0.5128,VRK,1.0,R,0.7057,...,8.483252,0.5128,GALAXY,1.0,2,0.5771,0.127512,dubious fit (negative doublet and weak lines)...,3,disagreement in flag 0 to 2. took one VI redsh...
246,35185741977882979,-1,0.33.0.dev3890,0,GALAXY,0.5128,ACE,1.0,--,--,...,8.483252,0.5128,GALAXY,1.0,2,0.5771,0.127512,dubious fit (negative doublet and weak lines)...,3,disagreement in flag 0 to 2. took one VI redsh...
296,35185741977882979,-1,0.33.0.dev3890,0,GALAXY,0.5128,CBA,1.0,--,--,...,8.483252,0.5128,GALAXY,1.0,2,0.5771,0.127512,dubious fit (negative doublet and weak lines)...,3,disagreement in flag 0 to 2. took one VI redsh...


conflict ID =  12


In [62]:
#copy this text to a new cell to display the conflict
conflict_id=13
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
151,35185747984122831,GALAXY,1.6296,VRK,2.0,--,1.6282,--,fit doublet corresponds to single strong spect...,1.6282,GALAXY,1.333333,2,1.364567,0.432322,fit doublet corresponds to single strong spect...,3,none
201,35185747984122831,GALAXY,1.6296,ACE,2.0,R,0.8359,GALAXY,Weak OII close to sky line only?,0.8359,GALAXY,1.333333,2,1.364567,0.432322,fit doublet corresponds to single strong spect...,3,none
250,35185747984122831,GALAXY,1.6296,CBA,0.0,--,--,--,--,1.6296,GALAXY,1.333333,2,1.364567,0.432322,fit doublet corresponds to single strong spect...,3,none


In [63]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8359
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. took one VI z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
151,35185747984122831,-1,0.33.0.dev3890,0,GALAXY,1.6296,VRK,1.0,--,1.6282,...,8.058549,0.8359,GALAXY,1.333333,2,1.364567,0.432322,fit doublet corresponds to single strong spect...,3,disagreement in flag 0 to 2. took one VI z wit...
201,35185747984122831,-1,0.33.0.dev3890,0,GALAXY,1.6296,ACE,1.0,R,0.8359,...,8.058549,0.8359,GALAXY,1.333333,2,1.364567,0.432322,fit doublet corresponds to single strong spect...,3,disagreement in flag 0 to 2. took one VI z wit...
250,35185747984122831,-1,0.33.0.dev3890,0,GALAXY,1.6296,CBA,1.0,--,--,...,8.058549,0.8359,GALAXY,1.333333,2,1.364567,0.432322,fit doublet corresponds to single strong spect...,3,disagreement in flag 0 to 2. took one VI z wit...


conflict ID =  13


In [66]:
#copy this text to a new cell to display the conflict
conflict_id = 14
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
165,35185747988316811,GALAXY,0.6141,VRK,4.0,--,--,--,ELG or BGS ?,0.6141,GALAXY,4.0,0,0.6141,0.0,ELG or BGS ?| Broad MgII and strong OIII| [OI...,3,none
215,35185747988316811,GALAXY,0.6141,ACE,4.0,--,0.6141,QSO,Broad MgII and strong OIII,0.6141,QSO,4.0,0,0.6141,0.0,ELG or BGS ?| Broad MgII and strong OIII| [OI...,3,none
264,35185747988316811,GALAXY,0.6141,CBA,4.0,--,--,--,"[OII]"","" H&K"","" Hb"","" [OIII]",0.6141,GALAXY,4.0,0,0.6141,0.0,ELG or BGS ?| Broad MgII and strong OIII| [OI...,3,none


In [67]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype. broad MgII so set to QSO'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
165,35185747988316811,-1,0.33.0.dev3890,0,GALAXY,0.6141,VRK,4.0,--,--,...,2731.733192,0.6141,QSO,4.0,0,0.6141,0.0,ELG or BGS ?| Broad MgII and strong OIII| [OI...,3,disagreement in spectype. broad MgII so set to...
215,35185747988316811,-1,0.33.0.dev3890,0,GALAXY,0.6141,ACE,4.0,--,0.6141,...,2731.733192,0.6141,QSO,4.0,0,0.6141,0.0,ELG or BGS ?| Broad MgII and strong OIII| [OI...,3,disagreement in spectype. broad MgII so set to...
264,35185747988316811,-1,0.33.0.dev3890,0,GALAXY,0.6141,CBA,4.0,--,--,...,2731.733192,0.6141,QSO,4.0,0,0.6141,0.0,ELG or BGS ?| Broad MgII and strong OIII| [OI...,3,disagreement in spectype. broad MgII so set to...


conflict ID =  14


In [68]:
#copy this text to a new cell to display the conflict
conflict_id = 15
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
178,35185747988318730,GALAXY,0.5481,VRK,0.0,R,--,--,dubious fit (negative doublet and OIII lines)...,0.5481,GALAXY,1.0,2,0.7123,0.318196,dubious fit (negative doublet and OIII lines)...,3,none
227,35185747988318730,GALAXY,0.5481,ACE,2.0,--,1.0407,GALAXY,Based on OII near sky line only,1.0407,GALAXY,1.0,2,0.7123,0.318196,dubious fit (negative doublet and OIII lines)...,3,none
277,35185747988318730,GALAXY,0.5481,CBA,1.0,--,--,--,--,0.5481,GALAXY,1.0,2,0.7123,0.318196,dubious fit (negative doublet and OIII lines)...,3,none


In [69]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0407
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag 0 to 2. set to VI z and flag 1. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
178,35185747988318730,-1,0.33.0.dev3890,0,GALAXY,0.5481,VRK,1.0,R,--,...,20.048147,1.0407,GALAXY,1.0,2,0.7123,0.318196,dubious fit (negative doublet and OIII lines)...,3,disagreement in z and flag 0 to 2. set to VI z...
227,35185747988318730,-1,0.33.0.dev3890,0,GALAXY,0.5481,ACE,1.0,--,1.0407,...,20.048147,1.0407,GALAXY,1.0,2,0.7123,0.318196,dubious fit (negative doublet and OIII lines)...,3,disagreement in z and flag 0 to 2. set to VI z...
277,35185747988318730,-1,0.33.0.dev3890,0,GALAXY,0.5481,CBA,1.0,--,--,...,20.048147,1.0407,GALAXY,1.0,2,0.7123,0.318196,dubious fit (negative doublet and OIII lines)...,3,disagreement in z and flag 0 to 2. set to VI z...


conflict ID =  15


In [72]:
#copy this text to a new cell to display the conflict
conflict_id=16
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
188,35185747988321049,GALAXY,0.1977,VRK,2.0,RS,0.6083,--,dubious fit (doublet in noise + blue/red slici...,0.6083,GALAXY,2.0,0,0.471533,0.343074,dubious fit (doublet in noise + blue/red slici...,3,none
237,35185747988321049,GALAXY,0.1977,ACE,2.0,R,0.6086,GALAXY,Based on OII only,0.6086,GALAXY,2.0,0,0.471533,0.343074,dubious fit (doublet in noise + blue/red slici...,3,none
287,35185747988321049,GALAXY,0.1977,CBA,2.0,S,--,--,Drop at 5800AA. Not sure of what the line at ...,0.1977,GALAXY,2.0,0,0.471533,0.343074,dubious fit (doublet in noise + blue/red slici...,3,none


In [73]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6083
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with flag 2. redrock z is incorrect, but OII not secure. take VI z with flag 2. calibration issues'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
188,35185747988321049,-1,0.33.0.dev3890,0,GALAXY,0.1977,VRK,2.0,RS,0.6083,...,76.877539,0.6083,GALAXY,2.0,0,0.471533,0.343074,dubious fit (doublet in noise + blue/red slici...,3,disagreement in z with flag 2. redrock z is in...
237,35185747988321049,-1,0.33.0.dev3890,0,GALAXY,0.1977,ACE,2.0,R,0.6086,...,76.877539,0.6083,GALAXY,2.0,0,0.471533,0.343074,dubious fit (doublet in noise + blue/red slici...,3,disagreement in z with flag 2. redrock z is in...
287,35185747988321049,-1,0.33.0.dev3890,0,GALAXY,0.1977,CBA,2.0,S,--,...,76.877539,0.6083,GALAXY,2.0,0,0.471533,0.343074,dubious fit (doublet in noise + blue/red slici...,3,disagreement in z with flag 2. redrock z is in...


conflict ID =  16


In [76]:
#copy this text to a new cell to display the conflict
conflict_id=17
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
189,35185747988321076,GALAXY,0.8452,VRK,2.0,--,--,--,small and broad doublet + Balmer absorption l...,0.8452,GALAXY,3.0,2,0.738033,0.211,small and broad doublet + Balmer absorption l...,3,none
238,35185747988321076,GALAXY,0.8452,ACE,3.0,R,0.5237,GALAXY,Clear OII missed by Redrock,0.5237,GALAXY,3.0,2,0.738033,0.211,small and broad doublet + Balmer absorption l...,3,none
288,35185747988321076,GALAXY,0.8452,CBA,4.0,--,--,--,"[OII]"","" H&K"","" Hb"","" continuum",0.8452,GALAXY,3.0,2,0.738033,0.211,small and broad doublet + Balmer absorption l...,3,none


In [77]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5237
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'redrock z fits OK, but clear OII doublet missed. possibly two objects. take OII z. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
189,35185747988321076,-1,0.33.0.dev3890,0,GALAXY,0.8452,VRK,3.0,--,--,...,108.510673,0.5237,GALAXY,3.0,2,0.738033,0.211,small and broad doublet + Balmer absorption l...,3,"redrock z fits OK, but clear OII doublet misse..."
238,35185747988321076,-1,0.33.0.dev3890,0,GALAXY,0.8452,ACE,3.0,R,0.5237,...,108.510673,0.5237,GALAXY,3.0,2,0.738033,0.211,small and broad doublet + Balmer absorption l...,3,"redrock z fits OK, but clear OII doublet misse..."
288,35185747988321076,-1,0.33.0.dev3890,0,GALAXY,0.8452,CBA,3.0,--,--,...,108.510673,0.5237,GALAXY,3.0,2,0.738033,0.211,small and broad doublet + Balmer absorption l...,3,"redrock z fits OK, but clear OII doublet misse..."


conflict ID =  17


In [80]:
#copy this text to a new cell to display the conflict
conflict_id=18
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
311,35185747992511204,GALAXY,0.9263,CBA,0.0,--,--,--,--,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,none
361,35185747992511204,GALAXY,0.9263,ACE,3.0,RC,--,STAR,Much more likely to be a star,0.9263,STAR,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,none
411,35185747992511204,GALAXY,0.9263,vrk,1.0,RS,0.9849,--,RR doublet not convincing + VI only doublet s...,0.9849,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,none


In [81]:
# copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY' 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, z and spectype. very low SNR. keep redrock answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
311,35185747992511204,-1,0.33.0.dev3890,0,GALAXY,0.9263,CBA,1.0,--,--,...,6.950417,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."
361,35185747992511204,-1,0.33.0.dev3890,0,GALAXY,0.9263,ACE,1.0,RC,--,...,6.950417,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."
411,35185747992511204,-1,0.33.0.dev3890,0,GALAXY,0.9263,vrk,1.0,RS,0.9849,...,6.950417,0.9849,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."


conflict ID =  18


In [84]:
#copy this text to a new cell to display the conflict
conflict_id = 19
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
316,35185747992511879,GALAXY,0.9567,CBA,1.0,--,--,--,--,0.9567,GALAXY,1.0,2,1.2384,0.431901,--| Based on CIII] only|noisy spectrum nothing...,3,none
366,35185747992511879,GALAXY,0.9567,ACE,2.0,R,1.8018,GALAXY,Based on CIII] only,1.8018,GALAXY,1.0,2,1.2384,0.431901,--| Based on CIII] only|noisy spectrum nothing...,3,none
416,35185747992511879,GALAXY,0.9567,vrk,0.0,RS,--,--,noisy spectrum nothing there,0.9567,GALAXY,1.0,2,1.2384,0.431901,--| Based on CIII] only|noisy spectrum nothing...,3,none


In [85]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.8018
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag 0 to 2. take VI z, flag 1. very low signal.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
316,35185747992511879,-1,0.33.0.dev3890,0,GALAXY,0.9567,CBA,1.0,--,--,...,2.87748,1.8018,GALAXY,1.0,2,1.2384,0.431901,--| Based on CIII] only|noisy spectrum nothing...,3,"disagreement in z and flag 0 to 2. take VI z, ..."
366,35185747992511879,-1,0.33.0.dev3890,0,GALAXY,0.9567,ACE,1.0,R,1.8018,...,2.87748,1.8018,GALAXY,1.0,2,1.2384,0.431901,--| Based on CIII] only|noisy spectrum nothing...,3,"disagreement in z and flag 0 to 2. take VI z, ..."
416,35185747992511879,-1,0.33.0.dev3890,0,GALAXY,0.9567,vrk,1.0,RS,--,...,2.87748,1.8018,GALAXY,1.0,2,1.2384,0.431901,--| Based on CIII] only|noisy spectrum nothing...,3,"disagreement in z and flag 0 to 2. take VI z, ..."


conflict ID =  19


In [90]:
#copy this text to a new cell to display the conflict
conflict_id = 20
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
320,35185747992512523,GALAXY,-0.0027,CBA,0.0,--,--,--,--,-0.0027,GALAXY,1.0,2,0.707467,1.245463,--| Based on weak OII only| nosiy spectrum + p...,3,none
370,35185747992512523,GALAXY,-0.0027,ACE,2.0,--,0.8857,GALAXY,Based on weak OII only,0.8857,GALAXY,1.0,2,0.707467,1.245463,--| Based on weak OII only| nosiy spectrum + p...,3,none
420,35185747992512523,GALAXY,-0.0027,vrk,1.0,R,1.2394,--,nosiy spectrum + possible doublet at VI redsh...,1.2394,GALAXY,1.0,2,0.707467,1.245463,--| Based on weak OII only| nosiy spectrum + p...,3,none


In [91]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8857
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag 0 to 2. very low SNR. take one VI z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
320,35185747992512523,-1,0.33.0.dev3890,0,GALAXY,-0.0027,CBA,1.0,--,--,...,3.52461,0.8857,GALAXY,1.0,2,0.707467,1.245463,--| Based on weak OII only| nosiy spectrum + p...,3,disagreement in z and flag 0 to 2. very low SN...
370,35185747992512523,-1,0.33.0.dev3890,0,GALAXY,-0.0027,ACE,1.0,--,0.8857,...,3.52461,0.8857,GALAXY,1.0,2,0.707467,1.245463,--| Based on weak OII only| nosiy spectrum + p...,3,disagreement in z and flag 0 to 2. very low SN...
420,35185747992512523,-1,0.33.0.dev3890,0,GALAXY,-0.0027,vrk,1.0,R,1.2394,...,3.52461,0.8857,GALAXY,1.0,2,0.707467,1.245463,--| Based on weak OII only| nosiy spectrum + p...,3,disagreement in z and flag 0 to 2. very low SN...


conflict ID =  20


In [97]:
#copy this text to a new cell to display the conflict
conflict_id = 21
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
327,35185747992514594,GALAXY,1.6047,CBA,1.0,--,--,--,--,1.6047,GALAXY,0.666667,1,1.426267,0.258674,--| No signal|RR doublet in skyline noise + VI...,3,none
377,35185747992514594,GALAXY,1.6047,ACE,0.0,--,--,--,No signal,1.6047,GALAXY,0.666667,1,1.426267,0.258674,--| No signal|RR doublet in skyline noise + VI...,3,none
427,35185747992514594,GALAXY,1.6047,vrk,1.0,R,1.0694,--,RR doublet in skyline noise + VI redshift prov...,1.0694,GALAXY,0.666667,1,1.426267,0.258674,--| No signal|RR doublet in skyline noise + VI...,3,none


In [96]:
print(vi.loc[427]['VI comment'])

RR doublet in skyline noise + VI redshift provides a possible weak doublet


In [98]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6047
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, flag 0 to 1. very low signal. take redrock z at flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
327,35185747992514594,-1,0.33.0.dev3890,0,GALAXY,1.6047,CBA,1.0,--,--,...,3.614973,1.6047,GALAXY,0.666667,1,1.426267,0.258674,--| No signal|RR doublet in skyline noise + VI...,3,"disagreement in z, flag 0 to 1. very low signa..."
377,35185747992514594,-1,0.33.0.dev3890,0,GALAXY,1.6047,ACE,1.0,--,--,...,3.614973,1.6047,GALAXY,0.666667,1,1.426267,0.258674,--| No signal|RR doublet in skyline noise + VI...,3,"disagreement in z, flag 0 to 1. very low signa..."
427,35185747992514594,-1,0.33.0.dev3890,0,GALAXY,1.6047,vrk,1.0,R,1.0694,...,3.614973,1.6047,GALAXY,0.666667,1,1.426267,0.258674,--| No signal|RR doublet in skyline noise + VI...,3,"disagreement in z, flag 0 to 1. very low signa..."


conflict ID =  21


In [101]:
#copy this text to a new cell to display the conflict
conflict_id = 22
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
334,35185747996705618,GALAXY,1.6302,CBA,1.0,--,--,--,--,1.6302,GALAXY,1.0,2,1.0868,1.6302,--| More probably a star with Halpha in emissi...,3,none
384,35185747996705618,GALAXY,1.6302,ACE,2.0,RC,0.0000,STAR,More probably a star with Halpha in emission,0.0000,STAR,1.0,2,1.0868,1.6302,--| More probably a star with Halpha in emissi...,3,none
434,35185747996705618,GALAXY,1.6302,vrk,0.0,R,--,--,insecure doublet in skyline noise region,1.6302,GALAXY,1.0,2,1.0868,1.6302,--| More probably a star with Halpha in emissi...,3,none


In [102]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6302
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, spectype and flag 0 to 2. take redrock answer, flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
334,35185747996705618,-1,0.33.0.dev3890,0,GALAXY,1.6302,CBA,1.0,--,--,...,17.322401,1.6302,GALAXY,1.0,2,1.0868,1.6302,--| More probably a star with Halpha in emissi...,3,"disagreement in z, spectype and flag 0 to 2. t..."
384,35185747996705618,-1,0.33.0.dev3890,0,GALAXY,1.6302,ACE,1.0,RC,0.0000,...,17.322401,1.6302,GALAXY,1.0,2,1.0868,1.6302,--| More probably a star with Halpha in emissi...,3,"disagreement in z, spectype and flag 0 to 2. t..."
434,35185747996705618,-1,0.33.0.dev3890,0,GALAXY,1.6302,vrk,1.0,R,--,...,17.322401,1.6302,GALAXY,1.0,2,1.0868,1.6302,--| More probably a star with Halpha in emissi...,3,"disagreement in z, spectype and flag 0 to 2. t..."


conflict ID =  22


In [105]:
#copy this text to a new cell to display the conflict
conflict_id = 23
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
302,35185747996705619,GALAXY,0.0962,CBA,1.0,--,--,--,No convincing fit,0.0962,GALAXY,1.333333,3,0.0962,0.0,No convincing fit| No signal|RR doublet in no...,3,none
352,35185747996705619,GALAXY,0.0962,ACE,0.0,--,--,--,No signal,0.0962,GALAXY,1.333333,3,0.0962,0.0,No convincing fit| No signal|RR doublet in no...,3,none
402,35185747996705619,GALAXY,0.0962,vrk,3.0,R,--,--,RR doublet in noise + more convincing Hbeta OI...,0.0962,GALAXY,1.333333,3,0.0962,0.0,No convincing fit| No signal|RR doublet in no...,3,none


In [106]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 3. one secure line, no continuum, maybe Ha but unsure. flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
302,35185747996705619,-1,0.33.0.dev3890,0,GALAXY,0.0962,CBA,2.0,--,--,...,32.196587,0.0962,GALAXY,1.333333,3,0.0962,0.0,No convincing fit| No signal|RR doublet in no...,3,"disagreement in flag 0 to 3. one secure line, ..."
352,35185747996705619,-1,0.33.0.dev3890,0,GALAXY,0.0962,ACE,2.0,--,--,...,32.196587,0.0962,GALAXY,1.333333,3,0.0962,0.0,No convincing fit| No signal|RR doublet in no...,3,"disagreement in flag 0 to 3. one secure line, ..."
402,35185747996705619,-1,0.33.0.dev3890,0,GALAXY,0.0962,vrk,2.0,R,--,...,32.196587,0.0962,GALAXY,1.333333,3,0.0962,0.0,No convincing fit| No signal|RR doublet in no...,3,"disagreement in flag 0 to 3. one secure line, ..."


conflict ID =  23


In [110]:
#copy this text to a new cell to display the conflict
conflict_id=24
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
339,35185747996707619,GALAXY,0.5095,CBA,1.0,S,--,--,Jump at 5800AA.,0.5095,GALAXY,1.666667,2,1.059233,0.546406,Jump at 5800AA.| Based on OII only. Very bad ...,3,none
389,35185747996707619,GALAXY,0.5095,ACE,3.0,RS,1.3343,GALAXY,Based on OII only. Very bad match between blu...,1.3343,GALAXY,1.666667,2,1.059233,0.546406,Jump at 5800AA.| Based on OII only. Very bad ...,3,none
439,35185747996707619,GALAXY,0.5095,vrk,1.0,RS,1.3339,--,calibration issue at 5800AA + VI redshift wou...,1.3339,GALAXY,1.666667,2,1.059233,0.546406,Jump at 5800AA.| Based on OII only. Very bad ...,3,none


In [108]:
print(vi.loc[389]['VI comment'])
print(vi.loc[439]['VI comment'])

 Based on OII only. Very bad match between blue and red arms
 calibration issue at 5800AA + VI redshift would give a possible weak doublet


In [112]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.3343
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'mismatch in z and flag, take VI z and the mean flag. calibration issue at 5800AA'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
339,35185747996707619,-1,0.33.0.dev3890,0,GALAXY,0.5095,CBA,2.0,S,--,...,12.163481,1.3343,GALAXY,1.666667,2,1.059233,0.546406,Jump at 5800AA.| Based on OII only. Very bad ...,3,"mismatch in z and flag, take VI z and the mean..."
389,35185747996707619,-1,0.33.0.dev3890,0,GALAXY,0.5095,ACE,2.0,RS,1.3343,...,12.163481,1.3343,GALAXY,1.666667,2,1.059233,0.546406,Jump at 5800AA.| Based on OII only. Very bad ...,3,"mismatch in z and flag, take VI z and the mean..."
439,35185747996707619,-1,0.33.0.dev3890,0,GALAXY,0.5095,vrk,2.0,RS,1.3339,...,12.163481,1.3343,GALAXY,1.666667,2,1.059233,0.546406,Jump at 5800AA.| Based on OII only. Very bad ...,3,"mismatch in z and flag, take VI z and the mean..."


conflict ID =  24


In [115]:
#copy this text to a new cell to display the conflict
conflict_id = 25
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
340,35185747996707757,GALAXY,1.6282,CBA,0.0,--,--,--,--,1.6282,GALAXY,1.333333,3,1.0721,1.409866,--| Based largely on OII in the blue| RR doubl...,3,none
390,35185747996707757,GALAXY,1.6282,ACE,3.0,R,0.0906,GALAXY,Based largely on OII in the blue,0.0906,GALAXY,1.333333,3,1.0721,1.409866,--| Based largely on OII in the blue| RR doubl...,3,none
440,35185747996707757,GALAXY,1.6282,vrk,1.0,--,1.4975,--,RR doublet in skyline nosie region + VI redsh...,1.4975,GALAXY,1.333333,3,1.0721,1.409866,--| Based largely on OII in the blue| RR doubl...,3,none


In [116]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.0906
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in VI z and flag, neither is convincing. Take one of them with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
340,35185747996707757,-1,0.33.0.dev3890,0,GALAXY,1.6282,CBA,2.0,--,--,...,13.56891,0.0906,GALAXY,1.333333,3,1.0721,1.409866,--| Based largely on OII in the blue| RR doubl...,3,"disagreement in VI z and flag, neither is conv..."
390,35185747996707757,-1,0.33.0.dev3890,0,GALAXY,1.6282,ACE,2.0,R,0.0906,...,13.56891,0.0906,GALAXY,1.333333,3,1.0721,1.409866,--| Based largely on OII in the blue| RR doubl...,3,"disagreement in VI z and flag, neither is conv..."
440,35185747996707757,-1,0.33.0.dev3890,0,GALAXY,1.6282,vrk,2.0,--,1.4975,...,13.56891,0.0906,GALAXY,1.333333,3,1.0721,1.409866,--| Based largely on OII in the blue| RR doubl...,3,"disagreement in VI z and flag, neither is conv..."


conflict ID =  25


In [124]:
#copy this text to a new cell to display the conflict
conflict_id = 26
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
341,35185747996708490,GALAXY,1.4081,CBA,4.0,--,--,--,[OII],1.4081,QSO,4.0,0,1.4081,0.0,[OII]| Very broad CIII] absorption?|--,3,disagreement in spectype due to possible broad...
391,35185747996708490,GALAXY,1.4081,ACE,4.0,--,1.4081,QSO,Very broad CIII] absorption?,1.4081,QSO,4.0,0,1.4081,0.0,[OII]| Very broad CIII] absorption?|--,3,disagreement in spectype due to possible broad...
441,35185747996708490,GALAXY,1.4081,vrk,4.0,--,--,--,--,1.4081,QSO,4.0,0,1.4081,0.0,[OII]| Very broad CIII] absorption?|--,3,disagreement in spectype due to possible broad...


In [128]:
#copy this text to a new cell to display the conflict
conflict_id= 27
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
307,35185747996708934,GALAXY,0.4922,CBA,1.0,--,--,--,--,0.4922,GALAXY,1.0,2,0.501667,0.019032,--| No signal| VI redshift gives slightly more...,3,none
357,35185747996708934,GALAXY,0.4922,ACE,0.0,--,--,--,No signal,0.4922,GALAXY,1.0,2,0.501667,0.019032,--| No signal| VI redshift gives slightly more...,3,none
407,35185747996708934,GALAXY,0.4922,vrk,2.0,R,0.5206,--,VI redshift gives slightly more convincing we...,0.5206,GALAXY,1.0,2,0.501667,0.019032,--| No signal| VI redshift gives slightly more...,3,none


In [127]:
print(vi.loc[ 407]['VI comment'])

 VI redshift gives slightly more convincing weak doublet + Hbeta and OIII


In [129]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5206
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag 0 to 2. take VI z with flag 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
307,35185747996708934,-1,0.33.0.dev3890,0,GALAXY,0.4922,CBA,1.0,--,--,...,2.073575,0.5206,GALAXY,1.0,2,0.501667,0.019032,--| No signal| VI redshift gives slightly more...,3,disagreement in z and flag 0 to 2. take VI z w...
357,35185747996708934,-1,0.33.0.dev3890,0,GALAXY,0.4922,ACE,1.0,--,--,...,2.073575,0.5206,GALAXY,1.0,2,0.501667,0.019032,--| No signal| VI redshift gives slightly more...,3,disagreement in z and flag 0 to 2. take VI z w...
407,35185747996708934,-1,0.33.0.dev3890,0,GALAXY,0.4922,vrk,1.0,R,0.5206,...,2.073575,0.5206,GALAXY,1.0,2,0.501667,0.019032,--| No signal| VI redshift gives slightly more...,3,disagreement in z and flag 0 to 2. take VI z w...


conflict ID =  27


In [136]:
#copy this text to a new cell to display the conflict
conflict_id= 28
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
343,35185747996709244,GALAXY,0.9089,CBA,1.0,--,--,--,--,0.9089,GALAXY,2.0,2,0.8663,0.071753,--| Based mainly on OII but| doublet Balmer ab...,3,none
393,35185747996709244,GALAXY,0.9089,ACE,3.0,R,0.7811,GALAXY,Based mainly on OII but,0.7811,GALAXY,2.0,2,0.8663,0.071753,--| Based mainly on OII but| doublet Balmer ab...,3,none
443,35185747996709244,GALAXY,0.9089,vrk,2.0,--,--,--,doublet Balmer absorption lines all weak + OI...,0.9089,GALAXY,2.0,2,0.8663,0.071753,--| Based mainly on OII but| doublet Balmer ab...,3,none


In [132]:
print(vi.loc[443]['VI comment'])

 doublet Balmer absorption lines all weak + OIII in noise


In [137]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7811
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. taking VI z based on OII, but not very secure. flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
343,35185747996709244,-1,0.33.0.dev3890,0,GALAXY,0.9089,CBA,2.0,--,--,...,4.276269,0.7811,GALAXY,2.0,2,0.8663,0.071753,--| Based mainly on OII but| doublet Balmer ab...,3,disagreement in z and flag. taking VI z based ...
393,35185747996709244,-1,0.33.0.dev3890,0,GALAXY,0.9089,ACE,2.0,R,0.7811,...,4.276269,0.7811,GALAXY,2.0,2,0.8663,0.071753,--| Based mainly on OII but| doublet Balmer ab...,3,disagreement in z and flag. taking VI z based ...
443,35185747996709244,-1,0.33.0.dev3890,0,GALAXY,0.9089,vrk,2.0,--,--,...,4.276269,0.7811,GALAXY,2.0,2,0.8663,0.071753,--| Based mainly on OII but| doublet Balmer ab...,3,disagreement in z and flag. taking VI z based ...


conflict ID =  28


In [140]:
#copy this text to a new cell to display the conflict
conflict_id=29
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
347,35185747996710287,GALAXY,0.9269,CBA,1.0,--,--,--,--,0.9269,GALAXY,1.0,2,0.964933,0.059214,--| No signal| weak doublet + Balmer lines not...,3,none
397,35185747996710287,GALAXY,0.9269,ACE,0.0,--,--,--,No signal,0.9269,GALAXY,1.0,2,0.964933,0.059214,--| No signal| weak doublet + Balmer lines not...,3,none
447,35185747996710287,GALAXY,0.9269,vrk,2.0,--,1.0410,--,weak doublet + Balmer lines not convincing + ...,1.0410,GALAXY,1.0,2,0.964933,0.059214,--| No signal| weak doublet + Balmer lines not...,3,none


In [141]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9269
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag 0 to 2. take redrock z, flag 1. '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
347,35185747996710287,-1,0.33.0.dev3890,0,GALAXY,0.9269,CBA,1.0,--,--,...,8.651826,0.9269,GALAXY,1.0,2,0.964933,0.059214,--| No signal| weak doublet + Balmer lines not...,3,disagreement in z and flag 0 to 2. take redroc...
397,35185747996710287,-1,0.33.0.dev3890,0,GALAXY,0.9269,ACE,1.0,--,--,...,8.651826,0.9269,GALAXY,1.0,2,0.964933,0.059214,--| No signal| weak doublet + Balmer lines not...,3,disagreement in z and flag 0 to 2. take redroc...
447,35185747996710287,-1,0.33.0.dev3890,0,GALAXY,0.9269,vrk,1.0,--,1.0410,...,8.651826,0.9269,GALAXY,1.0,2,0.964933,0.059214,--| No signal| weak doublet + Balmer lines not...,3,disagreement in z and flag 0 to 2. take redroc...


conflict ID =  29


In [144]:
#copy this text to a new cell to display the conflict
conflict_id= 30
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
309,35185748000899521,GALAXY,0.7242,CBA,1.0,--,--,--,--,0.7242,GALAXY,1.666667,1,0.816433,0.080327,--| Based on OII only| VI redshift would give ...,3,none
359,35185748000899521,GALAXY,0.7242,ACE,2.0,R,0.8624,--,Based on OII only,0.8624,GALAXY,1.666667,1,0.816433,0.080327,--| Based on OII only| VI redshift would give ...,3,none
409,35185748000899521,GALAXY,0.7242,vrk,2.0,R,0.8627,--,VI redshift would give doublet but very weak,0.8627,GALAXY,1.666667,1,0.816433,0.080327,--| Based on OII only| VI redshift would give ...,3,none


In [145]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8627
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. Take VI z based on weak OII, with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
309,35185748000899521,-1,0.33.0.dev3890,0,GALAXY,0.7242,CBA,2.0,--,--,...,4.101488,0.8627,GALAXY,1.666667,1,0.816433,0.080327,--| Based on OII only| VI redshift would give ...,3,disagreement in z and flag. Take VI z based on...
359,35185748000899521,-1,0.33.0.dev3890,0,GALAXY,0.7242,ACE,2.0,R,0.8624,...,4.101488,0.8627,GALAXY,1.666667,1,0.816433,0.080327,--| Based on OII only| VI redshift would give ...,3,disagreement in z and flag. Take VI z based on...
409,35185748000899521,-1,0.33.0.dev3890,0,GALAXY,0.7242,vrk,2.0,R,0.8627,...,4.101488,0.8627,GALAXY,1.666667,1,0.816433,0.080327,--| Based on OII only| VI redshift would give ...,3,disagreement in z and flag. Take VI z based on...


conflict ID =  30


In [148]:
#copy this text to a new cell to display the conflict
conflict_id= 31
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
453,35185748000900278,GALAXY,0.9747,BJW,1.0,--,--,--,"low S/N"","" weak [O II]",0.9747,GALAXY,2.0,2,0.9747,0.0,"low S/N"","" weak [O II]| weak doublet + Balmer...",2,none
503,35185748000900278,GALAXY,0.9747,vrk,3.0,--,--,--,weak doublet + Balmer lines,0.9747,GALAXY,2.0,2,0.9747,0.0,"low S/N"","" weak [O II]| weak doublet + Balmer...",2,none


In [149]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
453,35185748000900278,-1,0.33.0.dev3890,0,GALAXY,0.9747,BJW,2.0,--,--,...,8.757666,0.9747,GALAXY,2.0,2,0.9747,0.0,"low S/N"","" weak [O II]| weak doublet + Balmer...",2,"flag disagreement, take mean."
503,35185748000900278,-1,0.33.0.dev3890,0,GALAXY,0.9747,vrk,2.0,--,--,...,8.757666,0.9747,GALAXY,2.0,2,0.9747,0.0,"low S/N"","" weak [O II]| weak doublet + Balmer...",2,"flag disagreement, take mean."


conflict ID =  31


In [152]:
#copy this text to a new cell to display the conflict
conflict_id=32
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
460,35185748000901009,GALAXY,0.3536,BJW,0.0,R,--,--,--,0.3536,GALAXY,0.5,1,0.4386,0.125591,--| very weak features from RR fit + VI is an ...,2,none
510,35185748000901009,GALAXY,0.3536,vrk,1.0,--,0.5236,--,very weak features from RR fit + VI is an equ...,0.5236,GALAXY,0.5,1,0.4386,0.125591,--| very weak features from RR fit + VI is an ...,2,none


In [153]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.5236
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 0.5
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with flag 0 to 1. took VI z with mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
460,35185748000901009,-1,0.33.0.dev3890,0,GALAXY,0.3536,BJW,0.5,R,--,...,0.317996,0.5236,GALAXY,0.5,1,0.4386,0.125591,--| very weak features from RR fit + VI is an ...,2,disagreement in z with flag 0 to 1. took VI z ...
510,35185748000901009,-1,0.33.0.dev3890,0,GALAXY,0.3536,vrk,0.5,--,0.5236,...,0.317996,0.5236,GALAXY,0.5,1,0.4386,0.125591,--| very weak features from RR fit + VI is an ...,2,disagreement in z with flag 0 to 1. took VI z ...


conflict ID =  32


In [154]:
#copy this text to a new cell to display the conflict
conflict_id=33
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
463,35185748000901678,GALAXY,0.9559,BJW,0.0,R,--,--,no real features,0.9559,GALAXY,0.0,0,0.96135,0.005573,no real features| RR fit not convincing + bet...,2,none
513,35185748000901678,GALAXY,0.9559,vrk,0.0,R,0.9668,--,RR fit not convincing + better doublet from V...,0.9668,GALAXY,0.0,0,0.96135,0.005573,no real features| RR fit not convincing + bet...,2,none


In [155]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9559
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with flag 0. took redrock z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
463,35185748000901678,-1,0.33.0.dev3890,0,GALAXY,0.9559,BJW,0.0,R,--,...,13.118538,0.9559,GALAXY,0.0,0,0.96135,0.005573,no real features| RR fit not convincing + bet...,2,disagreement in z with flag 0. took redrock z.
513,35185748000901678,-1,0.33.0.dev3890,0,GALAXY,0.9559,vrk,0.0,R,0.9668,...,13.118538,0.9559,GALAXY,0.0,0,0.96135,0.005573,no real features| RR fit not convincing + bet...,2,disagreement in z with flag 0. took redrock z.


conflict ID =  33


In [156]:
#copy this text to a new cell to display the conflict
conflict_id=34
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
469,35185748000903455,GALAXY,0.6839,BJW,0.0,R,--,--,--,0.6839,GALAXY,0.5,1,0.6875,0.004276,--| RR fit not convincing + VI slightly better...,2,none
519,35185748000903455,GALAXY,0.6839,vrk,1.0,R,0.6911,--,RR fit not convincing + VI slightly better do...,0.6911,GALAXY,0.5,1,0.6875,0.004276,--| RR fit not convincing + VI slightly better...,2,none


In [157]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6839
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with flag 0 and 1. took redrock z with mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
469,35185748000903455,-1,0.33.0.dev3890,0,GALAXY,0.6839,BJW,0.0,R,--,...,15.62722,0.6839,GALAXY,0.5,1,0.6875,0.004276,--| RR fit not convincing + VI slightly better...,2,disagreement in z with flag 0 and 1. took redr...
519,35185748000903455,-1,0.33.0.dev3890,0,GALAXY,0.6839,vrk,1.0,R,0.6911,...,15.62722,0.6839,GALAXY,0.5,1,0.6875,0.004276,--| RR fit not convincing + VI slightly better...,2,disagreement in z with flag 0 and 1. took redr...


conflict ID =  34


In [158]:
#copy this text to a new cell to display the conflict
conflict_id=35
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
470,35185748000903650,GALAXY,0.7554,BJW,0.0,R,--,--,poor S/N,0.7554,GALAXY,0.0,0,0.8986,0.163154,poor S/N| RR fit not convincing + VI redshift...,2,none
520,35185748000903650,GALAXY,0.7554,vrk,0.0,--,1.0418,--,RR fit not convincing + VI redshift not bette...,1.0418,GALAXY,0.0,0,0.8986,0.163154,poor S/N| RR fit not convincing + VI redshift...,2,none


In [159]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7554
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with flag 0. took redrock z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
470,35185748000903650,-1,0.33.0.dev3890,0,GALAXY,0.7554,BJW,0.0,R,--,...,4.752329,0.7554,GALAXY,0.0,0,0.8986,0.163154,poor S/N| RR fit not convincing + VI redshift...,2,disagreement in z with flag 0. took redrock z.
520,35185748000903650,-1,0.33.0.dev3890,0,GALAXY,0.7554,vrk,0.0,--,1.0418,...,4.752329,0.7554,GALAXY,0.0,0,0.8986,0.163154,poor S/N| RR fit not convincing + VI redshift...,2,disagreement in z with flag 0. took redrock z.


conflict ID =  35


In [162]:
#copy this text to a new cell to display the conflict
conflict_id=36
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
479,35185748005094056,GALAXY,1.5548,BJW,2.0,--,--,--,[O II] partly on a sky line - real?,1.5548,GALAXY,3.0,2,1.5548,0.0,[O II] partly on a sky line - real?|--,2,none
529,35185748005094056,GALAXY,1.5548,vrk,4.0,--,--,--,--,1.5548,GALAXY,3.0,2,1.5548,0.0,[O II] partly on a sky line - real?|--,2,none


In [163]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but z not secure. set flag to 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
479,35185748005094056,-1,0.33.0.dev3890,0,GALAXY,1.5548,BJW,2.0,--,--,...,29.20702,1.5548,GALAXY,3.0,2,1.5548,0.0,[O II] partly on a sky line - real?|--,2,"disagreement in flag, but z not secure. set fl..."
529,35185748005094056,-1,0.33.0.dev3890,0,GALAXY,1.5548,vrk,2.0,--,--,...,29.20702,1.5548,GALAXY,3.0,2,1.5548,0.0,[O II] partly on a sky line - real?|--,2,"disagreement in flag, but z not secure. set fl..."


conflict ID =  36


In [166]:
#copy this text to a new cell to display the conflict
conflict_id= 37
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
486,35185748005095141,GALAXY,0.4026,BJW,0.0,--,--,--,too low S/N for absorption lines,0.4026,GALAXY,0.5,1,0.64645,0.347711,too low S/N for absorption lines|unsecure RR d...,2,none
536,35185748005095141,GALAXY,0.4026,vrk,1.0,R,0.8903,--,unsecure RR doublet and lines + VI solution no...,0.8903,GALAXY,0.5,1,0.64645,0.347711,too low S/N for absorption lines|unsecure RR d...,2,none


In [167]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.4026
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'z disagreement with flag 0 to 1. took redrock z with mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
486,35185748005095141,-1,0.33.0.dev3890,0,GALAXY,0.4026,BJW,0.0,--,--,...,4.842577,0.4026,GALAXY,0.5,1,0.64645,0.347711,too low S/N for absorption lines|unsecure RR d...,2,z disagreement with flag 0 to 1. took redrock ...
536,35185748005095141,-1,0.33.0.dev3890,0,GALAXY,0.4026,vrk,1.0,R,0.8903,...,4.842577,0.4026,GALAXY,0.5,1,0.64645,0.347711,too low S/N for absorption lines|unsecure RR d...,2,z disagreement with flag 0 to 1. took redrock ...


conflict ID =  37


In [170]:
#copy this text to a new cell to display the conflict
conflict_id= 38
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
491,35185748005095972,GALAXY,0.87,BJW,2.0,--,--,--,weak [O II],0.87,GALAXY,3.0,2,0.87,0.0,weak [O II]|--,2,none
541,35185748005095972,GALAXY,0.87,vrk,4.0,--,--,--,--,0.87,GALAXY,3.0,2,0.87,0.0,weak [O II]|--,2,none


In [171]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
491,35185748005095972,-1,0.33.0.dev3890,0,GALAXY,0.87,BJW,3.0,--,--,...,273.563801,0.87,GALAXY,3.0,2,0.87,0.0,weak [O II]|--,2,"flag disagreement, take mean"
541,35185748005095972,-1,0.33.0.dev3890,0,GALAXY,0.87,vrk,3.0,--,--,...,273.563801,0.87,GALAXY,3.0,2,0.87,0.0,weak [O II]|--,2,"flag disagreement, take mean"


conflict ID =  38


In [179]:
#copy this text to a new cell to display the conflict
conflict_id=39
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
495,35185748005096260,GALAXY,1.4987,BJW,1.0,--,--,--,Putative [O II] is right on sky lines,1.4987,GALAXY,1.5,1,1.4337,0.054882,Putative [O II] is right on sky lines| dubiou...,2,disagreement in z with low flag. take redrock ...
545,35185748005096260,GALAXY,1.4987,vrk,1.0,R,1.3687,--,dubious RR doublet in noisy region + VI doubl...,1.4987,GALAXY,1.5,1,1.4337,0.054882,Putative [O II] is right on sky lines| dubiou...,2,disagreement in z with low flag. take redrock ...


In [180]:
#copy this text to a new cell to display the conflict
conflict_id=40
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
550,35185748005097030,GALAXY,1.1643,SEM,1.0,R,0.9535,--,--,0.9535,GALAXY,0.666667,1,1.094033,0.107909,--|--| Noise,3,none
600,35185748005097030,GALAXY,1.1643,CGS,1.0,--,--,--,--,1.1643,GALAXY,0.666667,1,1.094033,0.107909,--|--| Noise,3,none
650,35185748005097030,GALAXY,1.1643,--,0.0,--,--,--,Noise,1.1643,GALAXY,0.666667,1,1.094033,0.107909,--|--| Noise,3,none


In [181]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1643
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'z disagreement with flag 1. took redrock z and mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
550,35185748005097030,-1,0.33.0.dev3890,0,GALAXY,1.1643,SEM,1.0,R,0.9535,...,2.243204,1.1643,GALAXY,0.666667,1,1.094033,0.107909,--|--| Noise,3,z disagreement with flag 1. took redrock z and...
600,35185748005097030,-1,0.33.0.dev3890,0,GALAXY,1.1643,CGS,1.0,--,--,...,2.243204,1.1643,GALAXY,0.666667,1,1.094033,0.107909,--|--| Noise,3,z disagreement with flag 1. took redrock z and...
650,35185748005097030,-1,0.33.0.dev3890,0,GALAXY,1.1643,--,0.0,--,--,...,2.243204,1.1643,GALAXY,0.666667,1,1.094033,0.107909,--|--| Noise,3,z disagreement with flag 1. took redrock z and...


conflict ID =  40


In [184]:
#copy this text to a new cell to display the conflict
conflict_id= 41
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
552,35185748005097490,GALAXY,1.6283,SEM,2.0,R,1.0337,--,possible balmer lines,1.0337,GALAXY,2.0,0,1.4301,0.292374,possible balmer lines|--|Possible OII signal ...,3,none
602,35185748005097490,GALAXY,1.6283,CGS,2.0,--,--,--,--,1.6283,GALAXY,2.0,0,1.4301,0.292374,possible balmer lines|--|Possible OII signal ...,3,none
652,35185748005097490,GALAXY,1.6283,--,2.0,--,--,--,Possible OII signal but no other discernible f...,1.6283,GALAXY,2.0,0,1.4301,0.292374,possible balmer lines|--|Possible OII signal ...,3,none


In [185]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.6283
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with low flag. took redrock z, flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
552,35185748005097490,-1,0.33.0.dev3890,0,GALAXY,1.6283,SEM,2.0,R,1.0337,...,34.387788,1.6283,GALAXY,2.0,0,1.4301,0.292374,possible balmer lines|--|Possible OII signal ...,3,disagreement in z with low flag. took redrock ...
602,35185748005097490,-1,0.33.0.dev3890,0,GALAXY,1.6283,CGS,2.0,--,--,...,34.387788,1.6283,GALAXY,2.0,0,1.4301,0.292374,possible balmer lines|--|Possible OII signal ...,3,disagreement in z with low flag. took redrock ...
652,35185748005097490,-1,0.33.0.dev3890,0,GALAXY,1.6283,--,2.0,--,--,...,34.387788,1.6283,GALAXY,2.0,0,1.4301,0.292374,possible balmer lines|--|Possible OII signal ...,3,disagreement in z with low flag. took redrock ...


conflict ID =  41


In [186]:
#copy this text to a new cell to display the conflict
conflict_id=42
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
557,35185748009288493,GALAXY,1.403,SEM,1.0,R,1.1658,--,--,1.1658,GALAXY,0.333333,1,1.323933,0.109521,--|--| Low signal to noise,3,none
607,35185748009288493,GALAXY,1.403,CGS,0.0,--,--,--,--,1.4030,GALAXY,0.333333,1,1.323933,0.109521,--|--| Low signal to noise,3,none
657,35185748009288493,GALAXY,1.403,--,0.0,--,--,--,Low signal to noise,1.4030,GALAXY,0.333333,1,1.323933,0.109521,--|--| Low signal to noise,3,none


In [187]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.403
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z with flag 0 to 1. take redrock z with mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
557,35185748009288493,-1,0.33.0.dev3890,0,GALAXY,1.403,SEM,1.0,R,1.1658,...,64.517504,1.403,GALAXY,0.333333,1,1.323933,0.109521,--|--| Low signal to noise,3,disagreement in z with flag 0 to 1. take redro...
607,35185748009288493,-1,0.33.0.dev3890,0,GALAXY,1.403,CGS,0.0,--,--,...,64.517504,1.403,GALAXY,0.333333,1,1.323933,0.109521,--|--| Low signal to noise,3,disagreement in z with flag 0 to 1. take redro...
657,35185748009288493,-1,0.33.0.dev3890,0,GALAXY,1.403,--,0.0,--,--,...,64.517504,1.403,GALAXY,0.333333,1,1.323933,0.109521,--|--| Low signal to noise,3,disagreement in z with flag 0 to 1. take redro...


conflict ID =  42


In [188]:
#copy this text to a new cell to display the conflict
conflict_id= 43
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
558,35185748009288692,GALAXY,1.5778,SEM,2.0,--,--,--,possible OII,1.5778,GALAXY,1.0,2,1.5778,0.0,possible OII|--|Noise,3,none
608,35185748009288692,GALAXY,1.5778,CGS,0.0,--,--,--,--,1.5778,GALAXY,1.0,2,1.5778,0.0,possible OII|--|Noise,3,none
658,35185748009288692,GALAXY,1.5778,--,1.0,--,--,--,Noise,1.5778,GALAXY,1.0,2,1.5778,0.0,possible OII|--|Noise,3,none


In [189]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement 0 to 2. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
558,35185748009288692,-1,0.33.0.dev3890,0,GALAXY,1.5778,SEM,1.0,--,--,...,26.669479,1.5778,GALAXY,1.0,2,1.5778,0.0,possible OII|--|Noise,3,flag disagreement 0 to 2. set to mean.
608,35185748009288692,-1,0.33.0.dev3890,0,GALAXY,1.5778,CGS,1.0,--,--,...,26.669479,1.5778,GALAXY,1.0,2,1.5778,0.0,possible OII|--|Noise,3,flag disagreement 0 to 2. set to mean.
658,35185748009288692,-1,0.33.0.dev3890,0,GALAXY,1.5778,--,1.0,--,--,...,26.669479,1.5778,GALAXY,1.0,2,1.5778,0.0,possible OII|--|Noise,3,flag disagreement 0 to 2. set to mean.


conflict ID =  43


In [192]:
#copy this text to a new cell to display the conflict
conflict_id= 44
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
572,35185748009288821,GALAXY,1.3539,SEM,4.0,--,--,--,OII and NeIII lines,1.3539,GALAXY,3.0,2,1.3539,0.0,OII and NeIII lines|--|Likely OII emission an...,3,none
622,35185748009288821,GALAXY,1.3539,CGS,2.0,--,--,--,--,1.3539,GALAXY,3.0,2,1.3539,0.0,OII and NeIII lines|--|Likely OII emission an...,3,none
672,35185748009288821,GALAXY,1.3539,--,3.0,--,1.3539,GALAXY,Likely OII emission and reasonable smoothed co...,1.3539,GALAXY,3.0,2,1.3539,0.0,OII and NeIII lines|--|Likely OII emission an...,3,none


In [193]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement from 2 to 4. OII compromised by noise spike, but still convincing. Take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
572,35185748009288821,-1,0.33.0.dev3890,0,GALAXY,1.3539,SEM,3.0,--,--,...,55.19361,1.3539,GALAXY,3.0,2,1.3539,0.0,OII and NeIII lines|--|Likely OII emission an...,3,flag disagreement from 2 to 4. OII compromised...
622,35185748009288821,-1,0.33.0.dev3890,0,GALAXY,1.3539,CGS,3.0,--,--,...,55.19361,1.3539,GALAXY,3.0,2,1.3539,0.0,OII and NeIII lines|--|Likely OII emission an...,3,flag disagreement from 2 to 4. OII compromised...
672,35185748009288821,-1,0.33.0.dev3890,0,GALAXY,1.3539,--,3.0,--,1.3539,...,55.19361,1.3539,GALAXY,3.0,2,1.3539,0.0,OII and NeIII lines|--|Likely OII emission an...,3,flag disagreement from 2 to 4. OII compromised...


conflict ID =  44


In [196]:
#copy this text to a new cell to display the conflict
conflict_id= 45
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
573,35185748009289720,GALAXY,0.7564,SEM,3.0,--,--,--,OII and balmer lines,0.7564,GALAXY,3.333333,1,0.7594,0.005124,OII and balmer lines|--| Probable classificat...,3,none
623,35185748009289720,GALAXY,0.7564,CGS,4.0,--,--,--,--,0.7564,GALAXY,3.333333,1,0.7594,0.005124,OII and balmer lines|--| Probable classificat...,3,none
673,35185748009289720,GALAXY,0.7564,--,3.0,--,0.7654,GALAXY,Probable classification unresolved OII and co...,0.7654,GALAXY,3.333333,1,0.7594,0.005124,OII and balmer lines|--| Probable classificat...,3,none


In [197]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.7564
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'z disagreement, i think a typo. set to redrock z, mean flag.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
573,35185748009289720,-1,0.33.0.dev3890,0,GALAXY,0.7564,SEM,3.0,--,--,...,168.143462,0.7564,GALAXY,3.333333,1,0.7594,0.005124,OII and balmer lines|--| Probable classificat...,3,"z disagreement, i think a typo. set to redrock..."
623,35185748009289720,-1,0.33.0.dev3890,0,GALAXY,0.7564,CGS,4.0,--,--,...,168.143462,0.7564,GALAXY,3.333333,1,0.7594,0.005124,OII and balmer lines|--| Probable classificat...,3,"z disagreement, i think a typo. set to redrock..."
673,35185748009289720,-1,0.33.0.dev3890,0,GALAXY,0.7564,--,3.0,--,0.7654,...,168.143462,0.7564,GALAXY,3.333333,1,0.7594,0.005124,OII and balmer lines|--| Probable classificat...,3,"z disagreement, i think a typo. set to redrock..."


conflict ID =  45


In [198]:
#copy this text to a new cell to display the conflict
conflict_id= 46
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
575,35185748009290088,GALAXY,1.3797,SEM,2.0,--,--,--,OII,1.3797,GALAXY,1.0,2,1.3797,0.0,OII|--| No strong signal,3,none
625,35185748009290088,GALAXY,1.3797,CGS,1.0,--,--,--,--,1.3797,GALAXY,1.0,2,1.3797,0.0,OII|--| No strong signal,3,none
675,35185748009290088,GALAXY,1.3797,--,0.0,--,--,--,No strong signal,1.3797,GALAXY,1.0,2,1.3797,0.0,OII|--| No strong signal,3,none


In [199]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement 0 to 2. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
575,35185748009290088,-1,0.33.0.dev3890,0,GALAXY,1.3797,SEM,1.0,--,--,...,36.440157,1.3797,GALAXY,1.0,2,1.3797,0.0,OII|--| No strong signal,3,flag disagreement 0 to 2. set to mean.
625,35185748009290088,-1,0.33.0.dev3890,0,GALAXY,1.3797,CGS,1.0,--,--,...,36.440157,1.3797,GALAXY,1.0,2,1.3797,0.0,OII|--| No strong signal,3,flag disagreement 0 to 2. set to mean.
675,35185748009290088,-1,0.33.0.dev3890,0,GALAXY,1.3797,--,1.0,--,--,...,36.440157,1.3797,GALAXY,1.0,2,1.3797,0.0,OII|--| No strong signal,3,flag disagreement 0 to 2. set to mean.


conflict ID =  46


In [202]:
#copy this text to a new cell to display the conflict
conflict_id= 47
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
563,35185748009290318,GALAXY,1.4102,SEM,4.0,--,--,--,OII and balmer lines,1.4102,GALAXY,3.0,2,1.4102,0.0,OII and balmer lines|--|Signal near 9000 with...,3,none
613,35185748009290318,GALAXY,1.4102,CGS,3.0,--,--,--,--,1.4102,GALAXY,3.0,2,1.4102,0.0,OII and balmer lines|--|Signal near 9000 with...,3,none
663,35185748009290318,GALAXY,1.4102,--,2.0,--,--,--,Signal near 9000 with no other features possib...,1.4102,GALAXY,3.0,2,1.4102,0.0,OII and balmer lines|--|Signal near 9000 with...,3,none


In [203]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, take the mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
563,35185748009290318,-1,0.33.0.dev3890,0,GALAXY,1.4102,SEM,3.0,--,--,...,178.280389,1.4102,GALAXY,3.0,2,1.4102,0.0,OII and balmer lines|--|Signal near 9000 with...,3,"disagreement in flag, take the mean."
613,35185748009290318,-1,0.33.0.dev3890,0,GALAXY,1.4102,CGS,3.0,--,--,...,178.280389,1.4102,GALAXY,3.0,2,1.4102,0.0,OII and balmer lines|--|Signal near 9000 with...,3,"disagreement in flag, take the mean."
663,35185748009290318,-1,0.33.0.dev3890,0,GALAXY,1.4102,--,3.0,--,--,...,178.280389,1.4102,GALAXY,3.0,2,1.4102,0.0,OII and balmer lines|--|Signal near 9000 with...,3,"disagreement in flag, take the mean."


conflict ID =  47


In [206]:
#copy this text to a new cell to display the conflict
conflict_id= 48
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
576,35185748009290333,GALAXY,1.2616,SEM,3.0,--,--,--,OII and weak balmer lines,1.2616,GALAXY,3.0,2,1.2616,0.0,OII and weak balmer lines|--| Strong OII,3,none
626,35185748009290333,GALAXY,1.2616,CGS,2.0,--,--,--,--,1.2616,GALAXY,3.0,2,1.2616,0.0,OII and weak balmer lines|--| Strong OII,3,none
676,35185748009290333,GALAXY,1.2616,--,4.0,--,1.2616,GALAXY,Strong OII,1.2616,GALAXY,3.0,2,1.2616,0.0,OII and weak balmer lines|--| Strong OII,3,none


In [207]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, take the mean'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
576,35185748009290333,-1,0.33.0.dev3890,0,GALAXY,1.2616,SEM,3.0,--,--,...,109.853837,1.2616,GALAXY,3.0,2,1.2616,0.0,OII and weak balmer lines|--| Strong OII,3,"disagreement in flag, take the mean"
626,35185748009290333,-1,0.33.0.dev3890,0,GALAXY,1.2616,CGS,3.0,--,--,...,109.853837,1.2616,GALAXY,3.0,2,1.2616,0.0,OII and weak balmer lines|--| Strong OII,3,"disagreement in flag, take the mean"
676,35185748009290333,-1,0.33.0.dev3890,0,GALAXY,1.2616,--,3.0,--,1.2616,...,109.853837,1.2616,GALAXY,3.0,2,1.2616,0.0,OII and weak balmer lines|--| Strong OII,3,"disagreement in flag, take the mean"


conflict ID =  48


In [208]:
#copy this text to a new cell to display the conflict
conflict_id=49
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
584,35185748009292017,GALAXY,1.4993,SEM,1.0,--,--,--,--,1.4993,GALAXY,1.0,2,1.4993,0.0,--|--|OII signal in high noise region,3,none
634,35185748009292017,GALAXY,1.4993,CGS,2.0,--,--,--,--,1.4993,GALAXY,1.0,2,1.4993,0.0,--|--|OII signal in high noise region,3,none
684,35185748009292017,GALAXY,1.4993,--,0.0,--,--,--,OII signal in high noise region,1.4993,GALAXY,1.0,2,1.4993,0.0,--|--|OII signal in high noise region,3,none


In [209]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. take the mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
584,35185748009292017,-1,0.33.0.dev3890,0,GALAXY,1.4993,SEM,1.0,--,--,...,82.442477,1.4993,GALAXY,1.0,2,1.4993,0.0,--|--|OII signal in high noise region,3,disagreement in flag 0 to 2. take the mean.
634,35185748009292017,-1,0.33.0.dev3890,0,GALAXY,1.4993,CGS,1.0,--,--,...,82.442477,1.4993,GALAXY,1.0,2,1.4993,0.0,--|--|OII signal in high noise region,3,disagreement in flag 0 to 2. take the mean.
684,35185748009292017,-1,0.33.0.dev3890,0,GALAXY,1.4993,--,1.0,--,--,...,82.442477,1.4993,GALAXY,1.0,2,1.4993,0.0,--|--|OII signal in high noise region,3,disagreement in flag 0 to 2. take the mean.


conflict ID =  49


In [212]:
#copy this text to a new cell to display the conflict
conflict_id=50
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
588,35185748013486472,GALAXY,1.3904,SEM,2.0,--,--,--,low SNR and possible OII,1.3904,GALAXY,3.0,2,1.3904,0.0,low SNR and possible OII|--| Weakly resolved ...,3,none
638,35185748013486472,GALAXY,1.3904,CGS,3.0,--,--,--,--,1.3904,GALAXY,3.0,2,1.3904,0.0,low SNR and possible OII|--| Weakly resolved ...,3,none
688,35185748013486472,GALAXY,1.3904,--,4.0,--,1.3904,GALAXY,Weakly resolved OII and continuum break,1.3904,GALAXY,3.0,2,1.3904,0.0,low SNR and possible OII|--| Weakly resolved ...,3,none


In [213]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. secure z, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
588,35185748013486472,-1,0.33.0.dev3890,0,GALAXY,1.3904,SEM,3.0,--,--,...,345.824798,1.3904,GALAXY,3.0,2,1.3904,0.0,low SNR and possible OII|--| Weakly resolved ...,3,"disagreement in flag. secure z, take mean."
638,35185748013486472,-1,0.33.0.dev3890,0,GALAXY,1.3904,CGS,3.0,--,--,...,345.824798,1.3904,GALAXY,3.0,2,1.3904,0.0,low SNR and possible OII|--| Weakly resolved ...,3,"disagreement in flag. secure z, take mean."
688,35185748013486472,-1,0.33.0.dev3890,0,GALAXY,1.3904,--,3.0,--,1.3904,...,345.824798,1.3904,GALAXY,3.0,2,1.3904,0.0,low SNR and possible OII|--| Weakly resolved ...,3,"disagreement in flag. secure z, take mean."


conflict ID =  50


In [216]:
#copy this text to a new cell to display the conflict
conflict_id=51
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
590,35185748017676546,GALAXY,1.0744,SEM,4.0,--,--,--,OII and Balmer lines,1.0744,GALAXY,3.0,3,1.0744,0.0,OII and Balmer lines|--| Resolved OII double ...,3,none
640,35185748017676546,GALAXY,1.0744,CGS,1.0,--,--,--,--,1.0744,GALAXY,3.0,3,1.0744,0.0,OII and Balmer lines|--| Resolved OII double ...,3,none
690,35185748017676546,GALAXY,1.0744,--,4.0,--,1.0744,GALAXY,Resolved OII double peak,1.0744,GALAXY,3.0,3,1.0744,0.0,OII and Balmer lines|--| Resolved OII double ...,3,none


In [217]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag mismatch, secure z. take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
590,35185748017676546,-1,0.33.0.dev3890,0,GALAXY,1.0744,SEM,3.0,--,--,...,135.911312,1.0744,GALAXY,3.0,3,1.0744,0.0,OII and Balmer lines|--| Resolved OII double ...,3,"flag mismatch, secure z. take mean."
640,35185748017676546,-1,0.33.0.dev3890,0,GALAXY,1.0744,CGS,3.0,--,--,...,135.911312,1.0744,GALAXY,3.0,3,1.0744,0.0,OII and Balmer lines|--| Resolved OII double ...,3,"flag mismatch, secure z. take mean."
690,35185748017676546,-1,0.33.0.dev3890,0,GALAXY,1.0744,--,3.0,--,1.0744,...,135.911312,1.0744,GALAXY,3.0,3,1.0744,0.0,OII and Balmer lines|--| Resolved OII double ...,3,"flag mismatch, secure z. take mean."


conflict ID =  51


In [218]:
#copy this text to a new cell to display the conflict
conflict_id=52
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
593,35185748017677288,GALAXY,1.1907,SEM,2.0,--,--,--,OII,1.1907,GALAXY,1.333333,2,1.1907,0.0,OII|--|No signal,3,none
643,35185748017677288,GALAXY,1.1907,CGS,2.0,--,--,--,--,1.1907,GALAXY,1.333333,2,1.1907,0.0,OII|--|No signal,3,none
693,35185748017677288,GALAXY,1.1907,--,0.0,--,--,--,No signal,1.1907,GALAXY,1.333333,2,1.1907,0.0,OII|--|No signal,3,none


In [219]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
593,35185748017677288,-1,0.33.0.dev3890,0,GALAXY,1.1907,SEM,1.333333,--,--,...,9.06647,1.1907,GALAXY,1.333333,2,1.1907,0.0,OII|--|No signal,3,disagreement in flag 0 to 2. set to mean.
643,35185748017677288,-1,0.33.0.dev3890,0,GALAXY,1.1907,CGS,1.333333,--,--,...,9.06647,1.1907,GALAXY,1.333333,2,1.1907,0.0,OII|--|No signal,3,disagreement in flag 0 to 2. set to mean.
693,35185748017677288,-1,0.33.0.dev3890,0,GALAXY,1.1907,--,1.333333,--,--,...,9.06647,1.1907,GALAXY,1.333333,2,1.1907,0.0,OII|--|No signal,3,disagreement in flag 0 to 2. set to mean.


conflict ID =  52


In [222]:
#copy this text to a new cell to display the conflict
conflict_id= 53
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
595,35185748017677969,GALAXY,0.9281,SEM,4.0,--,--,--,OII and Balmer lines,0.9281,GALAXY,4.0,0,0.9461,0.028007,OII and Balmer lines|--| High noise but resol...,3,none
645,35185748017677969,GALAXY,0.9281,CGS,4.0,--,--,--,--,0.9281,GALAXY,4.0,0,0.9461,0.028007,OII and Balmer lines|--| High noise but resol...,3,none
695,35185748017677969,GALAXY,0.9281,--,4.0,--,0.9821,GALAXY,High noise but resolved OII double peak,0.9821,GALAXY,4.0,0,0.9461,0.028007,OII and Balmer lines|--| High noise but resol...,3,none


In [223]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9281
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, i think a typo. redrock z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
595,35185748017677969,-1,0.33.0.dev3890,0,GALAXY,0.9281,SEM,4.0,--,--,...,270.719377,0.9281,GALAXY,4.0,0,0.9461,0.028007,OII and Balmer lines|--| High noise but resol...,3,"disagreement in z, i think a typo. redrock z i..."
645,35185748017677969,-1,0.33.0.dev3890,0,GALAXY,0.9281,CGS,4.0,--,--,...,270.719377,0.9281,GALAXY,4.0,0,0.9461,0.028007,OII and Balmer lines|--| High noise but resol...,3,"disagreement in z, i think a typo. redrock z i..."
695,35185748017677969,-1,0.33.0.dev3890,0,GALAXY,0.9281,--,4.0,--,0.9821,...,270.719377,0.9281,GALAXY,4.0,0,0.9461,0.028007,OII and Balmer lines|--| High noise but resol...,3,"disagreement in z, i think a typo. redrock z i..."


conflict ID =  53


In [226]:
#copy this text to a new cell to display the conflict
conflict_id= 54
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
596,35185748017680549,GALAXY,1.0437,SEM,4.0,--,--,--,OII and weak Balmer lines,1.0437,GALAXY,3.333333,2,1.0437,0.0,OII and weak Balmer lines|--| High noise but ...,3,none
646,35185748017680549,GALAXY,1.0437,CGS,2.0,--,--,--,--,1.0437,GALAXY,3.333333,2,1.0437,0.0,OII and weak Balmer lines|--| High noise but ...,3,none
696,35185748017680549,GALAXY,1.0437,--,4.0,--,1.0437,GALAXY,High noise but resolved OII double peak,1.0437,GALAXY,3.333333,2,1.0437,0.0,OII and weak Balmer lines|--| High noise but ...,3,none


In [227]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3.333333
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement, take mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
596,35185748017680549,-1,0.33.0.dev3890,0,GALAXY,1.0437,SEM,3.333333,--,--,...,18.499117,1.0437,GALAXY,3.333333,2,1.0437,0.0,OII and weak Balmer lines|--| High noise but ...,3,"flag disagreement, take mean."
646,35185748017680549,-1,0.33.0.dev3890,0,GALAXY,1.0437,CGS,3.333333,--,--,...,18.499117,1.0437,GALAXY,3.333333,2,1.0437,0.0,OII and weak Balmer lines|--| High noise but ...,3,"flag disagreement, take mean."
696,35185748017680549,-1,0.33.0.dev3890,0,GALAXY,1.0437,--,3.333333,--,1.0437,...,18.499117,1.0437,GALAXY,3.333333,2,1.0437,0.0,OII and weak Balmer lines|--| High noise but ...,3,"flag disagreement, take mean."


conflict ID =  54


In [230]:
#copy this text to a new cell to display the conflict
conflict_id=55
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
597,35185748021870953,GALAXY,0.8765,SEM,4.0,--,--,--,OIII and weak Balmer lines,0.8765,GALAXY,1.666667,4,0.8765,0.0,OIII and weak Balmer lines|--| Possible signa...,3,none
647,35185748021870953,GALAXY,0.8765,CGS,0.0,--,--,--,--,0.8765,GALAXY,1.666667,4,0.8765,0.0,OIII and weak Balmer lines|--| Possible signa...,3,none
697,35185748021870953,GALAXY,0.8765,--,1.0,--,--,--,Possible signal with very high noise,0.8765,GALAXY,1.666667,4,0.8765,0.0,OIII and weak Balmer lines|--| Possible signa...,3,none


In [231]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. z not secure, set flag to 1.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
597,35185748021870953,-1,0.33.0.dev3890,0,GALAXY,0.8765,SEM,1.0,--,--,...,3.255511,0.8765,GALAXY,1.666667,4,0.8765,0.0,OIII and weak Balmer lines|--| Possible signa...,3,"disagreement in flag. z not secure, set flag t..."
647,35185748021870953,-1,0.33.0.dev3890,0,GALAXY,0.8765,CGS,1.0,--,--,...,3.255511,0.8765,GALAXY,1.666667,4,0.8765,0.0,OIII and weak Balmer lines|--| Possible signa...,3,"disagreement in flag. z not secure, set flag t..."
697,35185748021870953,-1,0.33.0.dev3890,0,GALAXY,0.8765,--,1.0,--,--,...,3.255511,0.8765,GALAXY,1.666667,4,0.8765,0.0,OIII and weak Balmer lines|--| Possible signa...,3,"disagreement in flag. z not secure, set flag t..."


conflict ID =  55


In [236]:
#copy this text to a new cell to display the conflict
conflict_id= 56
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
569,35185754019725749,GALAXY,0.5235,SEM,1.0,RS,--,--,--,0.5235,GALAXY,1.666667,1,0.697833,0.343288,--|--| Discontinuity at 5800 signal at 7300 po...,3,none
619,35185754019725749,GALAXY,0.5235,CGS,2.0,--,--,--,--,0.5235,GALAXY,1.666667,1,0.697833,0.343288,--|--| Discontinuity at 5800 signal at 7300 po...,3,none
669,35185754019725749,GALAXY,0.5235,--,2.0,RS,1.0465,GALAXY,Discontinuity at 5800 signal at 7300 possible...,1.0465,GALAXY,1.666667,1,0.697833,0.343288,--|--| Discontinuity at 5800 signal at 7300 po...,3,none


In [237]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0465
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. taking VI redshift, flag 3, based on doublet. bad reduction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
569,35185754019725749,-1,0.33.0.dev3890,0,GALAXY,0.5235,SEM,3.0,RS,--,...,159.607862,1.0465,GALAXY,1.666667,1,0.697833,0.343288,--|--| Discontinuity at 5800 signal at 7300 po...,3,"disagreement in z. taking VI redshift, flag 3,..."
619,35185754019725749,-1,0.33.0.dev3890,0,GALAXY,0.5235,CGS,3.0,--,--,...,159.607862,1.0465,GALAXY,1.666667,1,0.697833,0.343288,--|--| Discontinuity at 5800 signal at 7300 po...,3,"disagreement in z. taking VI redshift, flag 3,..."
669,35185754019725749,-1,0.33.0.dev3890,0,GALAXY,0.5235,--,3.0,RS,1.0465,...,159.607862,1.0465,GALAXY,1.666667,1,0.697833,0.343288,--|--| Discontinuity at 5800 signal at 7300 po...,3,"disagreement in z. taking VI redshift, flag 3,..."


conflict ID =  56


In [240]:
#copy this text to a new cell to display the conflict
conflict_id= 57
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
721,35185754019727456,GALAXY,0.473,SEM,0.0,S,--,--,--,0.4730,GALAXY,2.0,4,0.641533,0.343245,--|--|--,3,none
771,35185754019727456,GALAXY,0.473,ASC,4.0,RS,0.9786,GALAXY,--,0.9786,GALAXY,2.0,4,0.641533,0.343245,--|--|--,3,none
821,35185754019727456,GALAXY,0.473,CGS,2.0,--,--,--,--,0.4730,GALAXY,2.0,4,0.641533,0.343245,--|--|--,3,none


In [241]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9786
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag and z. taking VI z, secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
721,35185754019727456,-1,0.33.0.dev3890,0,GALAXY,0.473,SEM,3.0,S,--,...,88.650926,0.9786,GALAXY,2.0,4,0.641533,0.343245,--|--|--,3,"disagreement in flag and z. taking VI z, secure."
771,35185754019727456,-1,0.33.0.dev3890,0,GALAXY,0.473,ASC,3.0,RS,0.9786,...,88.650926,0.9786,GALAXY,2.0,4,0.641533,0.343245,--|--|--,3,"disagreement in flag and z. taking VI z, secure."
821,35185754019727456,-1,0.33.0.dev3890,0,GALAXY,0.473,CGS,3.0,--,--,...,88.650926,0.9786,GALAXY,2.0,4,0.641533,0.343245,--|--|--,3,"disagreement in flag and z. taking VI z, secure."


conflict ID =  57


In [244]:
#copy this text to a new cell to display the conflict
conflict_id= 58
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
723,35185754019727934,QSO,3.9752,SEM,1.0,R,--,--,--,3.9752,QSO,1.0,2,3.9752,0.0,--|Same issue as spectrum 1|--,3,none
773,35185754019727934,QSO,3.9752,ASC,0.0,S,--,--,Same issue as spectrum 1,3.9752,QSO,1.0,2,3.9752,0.0,--|Same issue as spectrum 1|--,3,none
823,35185754019727934,QSO,3.9752,CGS,2.0,--,--,--,--,3.9752,QSO,1.0,2,3.9752,0.0,--|Same issue as spectrum 1|--,3,none


In [246]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement 0 to 2, set to mean. subtraction issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
723,35185754019727934,-1,0.33.0.dev3890,0,QSO,3.9752,SEM,1.0,R,--,...,51.994185,3.9752,QSO,1.0,2,3.9752,0.0,--|Same issue as spectrum 1|--,3,"flag disagreement 0 to 2, set to mean. subtrac..."
773,35185754019727934,-1,0.33.0.dev3890,0,QSO,3.9752,ASC,1.0,S,--,...,51.994185,3.9752,QSO,1.0,2,3.9752,0.0,--|Same issue as spectrum 1|--,3,"flag disagreement 0 to 2, set to mean. subtrac..."
823,35185754019727934,-1,0.33.0.dev3890,0,QSO,3.9752,CGS,1.0,--,--,...,51.994185,3.9752,QSO,1.0,2,3.9752,0.0,--|Same issue as spectrum 1|--,3,"flag disagreement 0 to 2, set to mean. subtrac..."


conflict ID =  58


In [249]:
#copy this text to a new cell to display the conflict
conflict_id= 59
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
724,35185754019728103,QSO,3.9991,SEM,1.0,R,--,--,--,3.9991,QSO,2.0,2,3.0342,1.375546,--|Same issue as spectrum 1|--,3,none
774,35185754019728103,QSO,3.9991,ASC,3.0,RCS,1.1044,GALAXY,Same issue as spectrum 1,1.1044,GALAXY,2.0,2,3.0342,1.375546,--|Same issue as spectrum 1|--,3,none
824,35185754019728103,QSO,3.9991,CGS,2.0,--,--,--,--,3.9991,QSO,2.0,2,3.0342,1.375546,--|Same issue as spectrum 1|--,3,none


In [250]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1044
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z, flag and spectype. issues in calibration. VI z secure by OII.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
724,35185754019728103,-1,0.33.0.dev3890,0,QSO,3.9991,SEM,3.0,R,--,...,22.634941,1.1044,GALAXY,2.0,2,3.0342,1.375546,--|Same issue as spectrum 1|--,3,"disagreement in z, flag and spectype. issues i..."
774,35185754019728103,-1,0.33.0.dev3890,0,QSO,3.9991,ASC,3.0,RCS,1.1044,...,22.634941,1.1044,GALAXY,2.0,2,3.0342,1.375546,--|Same issue as spectrum 1|--,3,"disagreement in z, flag and spectype. issues i..."
824,35185754019728103,-1,0.33.0.dev3890,0,QSO,3.9991,CGS,3.0,--,--,...,22.634941,1.1044,GALAXY,2.0,2,3.0342,1.375546,--|Same issue as spectrum 1|--,3,"disagreement in z, flag and spectype. issues i..."


conflict ID =  59


In [253]:
#copy this text to a new cell to display the conflict
conflict_id= 60
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
726,35185754019728346,QSO,3.9325,SEM,1.0,--,--,--,--,3.9325,QSO,1.333333,3,3.9325,0.0,--|Same issue as spectrum 1|--,3,none
776,35185754019728346,QSO,3.9325,ASC,0.0,CS,--,--,Same issue as spectrum 1,3.9325,QSO,1.333333,3,3.9325,0.0,--|Same issue as spectrum 1|--,3,none
826,35185754019728346,QSO,3.9325,CGS,3.0,--,--,--,--,3.9325,QSO,1.333333,3,3.9325,0.0,--|Same issue as spectrum 1|--,3,none


In [254]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement. redrock z not secure. driven by bad reduction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
726,35185754019728346,-1,0.33.0.dev3890,0,QSO,3.9325,SEM,1.0,--,--,...,82.687104,3.9325,QSO,1.333333,3,3.9325,0.0,--|Same issue as spectrum 1|--,3,flag disagreement. redrock z not secure. drive...
776,35185754019728346,-1,0.33.0.dev3890,0,QSO,3.9325,ASC,1.0,CS,--,...,82.687104,3.9325,QSO,1.333333,3,3.9325,0.0,--|Same issue as spectrum 1|--,3,flag disagreement. redrock z not secure. drive...
826,35185754019728346,-1,0.33.0.dev3890,0,QSO,3.9325,CGS,1.0,--,--,...,82.687104,3.9325,QSO,1.333333,3,3.9325,0.0,--|Same issue as spectrum 1|--,3,flag disagreement. redrock z not secure. drive...


conflict ID =  60


In [257]:
#copy this text to a new cell to display the conflict
conflict_id= 61
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
727,35185754019728524,GALAXY,0.7929,SEM,0.0,S,--,--,--,0.7929,GALAXY,2.666667,4,0.7929,0.0,--|Same issue as spectrum 1|--,3,none
777,35185754019728524,GALAXY,0.7929,ASC,4.0,S,--,--,Same issue as spectrum 1,0.7929,GALAXY,2.666667,4,0.7929,0.0,--|Same issue as spectrum 1|--,3,none
827,35185754019728524,GALAXY,0.7929,CGS,4.0,--,--,--,--,0.7929,GALAXY,2.666667,4,0.7929,0.0,--|Same issue as spectrum 1|--,3,none


In [259]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag but z is secure. extraction issues.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
727,35185754019728524,-1,0.33.0.dev3890,0,GALAXY,0.7929,SEM,3.0,S,--,...,33.168152,0.7929,GALAXY,2.666667,4,0.7929,0.0,--|Same issue as spectrum 1|--,3,disagreement in flag but z is secure. extracti...
777,35185754019728524,-1,0.33.0.dev3890,0,GALAXY,0.7929,ASC,3.0,S,--,...,33.168152,0.7929,GALAXY,2.666667,4,0.7929,0.0,--|Same issue as spectrum 1|--,3,disagreement in flag but z is secure. extracti...
827,35185754019728524,-1,0.33.0.dev3890,0,GALAXY,0.7929,CGS,3.0,--,--,...,33.168152,0.7929,GALAXY,2.666667,4,0.7929,0.0,--|Same issue as spectrum 1|--,3,disagreement in flag but z is secure. extracti...


conflict ID =  61


In [262]:
#copy this text to a new cell to display the conflict
conflict_id= 62
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
728,35185754019728782,QSO,4.0073,SEM,1.0,C,--,--,--,4.0073,QSO,2.0,2,4.0073,0.0,--|Same issue again. Red arm problem.|--,3,none
778,35185754019728782,QSO,4.0073,ASC,3.0,RCS,4.0073,GALAXY,Same issue again. Red arm problem.,4.0073,GALAXY,2.0,2,4.0073,0.0,--|Same issue again. Red arm problem.|--,3,none
828,35185754019728782,QSO,4.0073,CGS,2.0,--,--,--,--,4.0073,QSO,2.0,2,4.0073,0.0,--|Same issue again. Red arm problem.|--,3,none


In [263]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1069
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'issue with extraction, disagreement in flag. tentative VI redshift with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
728,35185754019728782,-1,0.33.0.dev3890,0,QSO,4.0073,SEM,2.0,C,--,...,26.380767,1.1069,GALAXY,2.0,2,4.0073,0.0,--|Same issue again. Red arm problem.|--,3,"issue with extraction, disagreement in flag. t..."
778,35185754019728782,-1,0.33.0.dev3890,0,QSO,4.0073,ASC,2.0,RCS,4.0073,...,26.380767,1.1069,GALAXY,2.0,2,4.0073,0.0,--|Same issue again. Red arm problem.|--,3,"issue with extraction, disagreement in flag. t..."
828,35185754019728782,-1,0.33.0.dev3890,0,QSO,4.0073,CGS,2.0,--,--,...,26.380767,1.1069,GALAXY,2.0,2,4.0073,0.0,--|Same issue again. Red arm problem.|--,3,"issue with extraction, disagreement in flag. t..."


conflict ID =  62


In [264]:
#copy this text to a new cell to display the conflict
conflict_id= 63
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
730,35185754019729132,GALAXY,0.6647,SEM,1.0,RC,--,--,--,0.6647,GALAXY,1.0,2,0.6647,0.0,--|Same issue as spectrum 29?|--,3,none
780,35185754019729132,GALAXY,0.6647,ASC,0.0,S,--,--,Same issue as spectrum 29?,0.6647,GALAXY,1.0,2,0.6647,0.0,--|Same issue as spectrum 29?|--,3,none
830,35185754019729132,GALAXY,0.6647,CGS,2.0,--,--,--,--,0.6647,GALAXY,1.0,2,0.6647,0.0,--|Same issue as spectrum 29?|--,3,none


In [265]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'flag disagreement 0 to 2. set to mean'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
730,35185754019729132,-1,0.33.0.dev3890,0,GALAXY,0.6647,SEM,1.0,RC,--,...,5.916432,0.6647,GALAXY,1.0,2,0.6647,0.0,--|Same issue as spectrum 29?|--,3,flag disagreement 0 to 2. set to mean
780,35185754019729132,-1,0.33.0.dev3890,0,GALAXY,0.6647,ASC,1.0,S,--,...,5.916432,0.6647,GALAXY,1.0,2,0.6647,0.0,--|Same issue as spectrum 29?|--,3,flag disagreement 0 to 2. set to mean
830,35185754019729132,-1,0.33.0.dev3890,0,GALAXY,0.6647,CGS,1.0,--,--,...,5.916432,0.6647,GALAXY,1.0,2,0.6647,0.0,--|Same issue as spectrum 29?|--,3,flag disagreement 0 to 2. set to mean


conflict ID =  63


In [268]:
#copy this text to a new cell to display the conflict
conflict_id= 64
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
731,35185754019729695,QSO,3.9908,SEM,1.0,RC,--,--,--,3.9908,QSO,1.666667,2,2.9575,1.639378,--|Same issue as spectrum 29? Looks like at le...,3,none
781,35185754019729695,QSO,3.9908,ASC,3.0,RCS,0.8909,GALAXY,Same issue as spectrum 29? Looks like at least...,0.8909,GALAXY,1.666667,2,2.9575,1.639378,--|Same issue as spectrum 29? Looks like at le...,3,none
831,35185754019729695,QSO,3.9908,CGS,1.0,--,--,--,--,3.9908,QSO,1.666667,2,2.9575,1.639378,--|Same issue as spectrum 29? Looks like at le...,3,none


In [269]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8909
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. redrock z wrong due to issues in reduction. take VI redshift, flag 3.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
731,35185754019729695,-1,0.33.0.dev3890,0,QSO,3.9908,SEM,3.0,RC,--,...,15.789654,0.8909,GALAXY,1.666667,2,2.9575,1.639378,--|Same issue as spectrum 29? Looks like at le...,3,disagreement in flag. redrock z wrong due to i...
781,35185754019729695,-1,0.33.0.dev3890,0,QSO,3.9908,ASC,3.0,RCS,0.8909,...,15.789654,0.8909,GALAXY,1.666667,2,2.9575,1.639378,--|Same issue as spectrum 29? Looks like at le...,3,disagreement in flag. redrock z wrong due to i...
831,35185754019729695,-1,0.33.0.dev3890,0,QSO,3.9908,CGS,3.0,--,--,...,15.789654,0.8909,GALAXY,1.666667,2,2.9575,1.639378,--|Same issue as spectrum 29? Looks like at le...,3,disagreement in flag. redrock z wrong due to i...


conflict ID =  64


In [272]:
#copy this text to a new cell to display the conflict
conflict_id= 65
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
732,35185754019729968,GALAXY,0.6351,SEM,1.0,RC,--,--,--,0.6351,GALAXY,2.0,3,0.822167,0.343221,--|Same issue as spectrum 29|--,3,none
782,35185754019729968,GALAXY,0.6351,ASC,4.0,RS,1.1963,GALAXY,Same issue as spectrum 29,1.1963,GALAXY,2.0,3,0.822167,0.343221,--|Same issue as spectrum 29|--,3,none
832,35185754019729968,GALAXY,0.6351,CGS,1.0,--,--,--,--,0.6351,GALAXY,2.0,3,0.822167,0.343221,--|Same issue as spectrum 29|--,3,none


In [274]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.1963
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. VI z secure. Issues in reduction.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
732,35185754019729968,-1,0.33.0.dev3890,0,GALAXY,0.6351,SEM,4.0,RC,--,...,255.827918,1.1963,GALAXY,2.0,3,0.822167,0.343221,--|Same issue as spectrum 29|--,3,disagreement in z and flag. VI z secure. Issue...
782,35185754019729968,-1,0.33.0.dev3890,0,GALAXY,0.6351,ASC,4.0,RS,1.1963,...,255.827918,1.1963,GALAXY,2.0,3,0.822167,0.343221,--|Same issue as spectrum 29|--,3,disagreement in z and flag. VI z secure. Issue...
832,35185754019729968,-1,0.33.0.dev3890,0,GALAXY,0.6351,CGS,4.0,--,--,...,255.827918,1.1963,GALAXY,2.0,3,0.822167,0.343221,--|Same issue as spectrum 29|--,3,disagreement in z and flag. VI z secure. Issue...


conflict ID =  65


In [277]:
#copy this text to a new cell to display the conflict
conflict_id= 66
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
733,35185754019730045,GALAXY,1.4203,SEM,1.0,--,--,--,--,1.4203,GALAXY,2.333333,3,1.4203,0.0,--|Same issue as spectrum 29|--,3,none
783,35185754019730045,GALAXY,1.4203,ASC,4.0,S,--,--,Same issue as spectrum 29,1.4203,GALAXY,2.333333,3,1.4203,0.0,--|Same issue as spectrum 29|--,3,none
833,35185754019730045,GALAXY,1.4203,CGS,2.0,--,--,--,--,1.4203,GALAXY,2.333333,3,1.4203,0.0,--|Same issue as spectrum 29|--,3,none


In [278]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. issues in calibration. z is good.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
733,35185754019730045,-1,0.33.0.dev3890,0,GALAXY,1.4203,SEM,3.0,--,--,...,249.717173,1.4203,GALAXY,2.333333,3,1.4203,0.0,--|Same issue as spectrum 29|--,3,disagreement in flag. issues in calibration. z...
783,35185754019730045,-1,0.33.0.dev3890,0,GALAXY,1.4203,ASC,3.0,S,--,...,249.717173,1.4203,GALAXY,2.333333,3,1.4203,0.0,--|Same issue as spectrum 29|--,3,disagreement in flag. issues in calibration. z...
833,35185754019730045,-1,0.33.0.dev3890,0,GALAXY,1.4203,CGS,3.0,--,--,...,249.717173,1.4203,GALAXY,2.333333,3,1.4203,0.0,--|Same issue as spectrum 29|--,3,disagreement in flag. issues in calibration. z...


conflict ID =  66


In [281]:
#copy this text to a new cell to display the conflict
conflict_id= 67
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
734,35185754019730170,GALAXY,0.7627,SEM,1.0,R,--,--,--,0.7627,GALAXY,3.0,3,0.7627,0.0,--|Same issue as spectrum 29?|--,3,none
784,35185754019730170,GALAXY,0.7627,ASC,4.0,--,--,--,Same issue as spectrum 29?,0.7627,GALAXY,3.0,3,0.7627,0.0,--|Same issue as spectrum 29?|--,3,none
834,35185754019730170,GALAXY,0.7627,CGS,4.0,--,--,--,--,0.7627,GALAXY,3.0,3,0.7627,0.0,--|Same issue as spectrum 29?|--,3,none


In [282]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, but z is secure. issues in calibration.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
734,35185754019730170,-1,0.33.0.dev3890,0,GALAXY,0.7627,SEM,4.0,R,--,...,191.783724,0.7627,GALAXY,3.0,3,0.7627,0.0,--|Same issue as spectrum 29?|--,3,"disagreement in flag, but z is secure. issues ..."
784,35185754019730170,-1,0.33.0.dev3890,0,GALAXY,0.7627,ASC,4.0,--,--,...,191.783724,0.7627,GALAXY,3.0,3,0.7627,0.0,--|Same issue as spectrum 29?|--,3,"disagreement in flag, but z is secure. issues ..."
834,35185754019730170,-1,0.33.0.dev3890,0,GALAXY,0.7627,CGS,4.0,--,--,...,191.783724,0.7627,GALAXY,3.0,3,0.7627,0.0,--|Same issue as spectrum 29?|--,3,"disagreement in flag, but z is secure. issues ..."


conflict ID =  67


In [285]:
#copy this text to a new cell to display the conflict
conflict_id = 68
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
700,35185754023920071,QSO,2.9698,SEM,1.0,C,--,--,--,2.9698,QSO,2.0,3,2.3731,0.82126,--|Strong absorption feature. RR falsely confi...,3,none
750,35185754023920071,QSO,2.9698,ASC,4.0,RCS,1.1797,GALAXY,Strong absorption feature. RR falsely confident,1.1797,GALAXY,2.0,3,2.3731,0.82126,--|Strong absorption feature. RR falsely confi...,3,none
800,35185754023920071,QSO,2.9698,CGS,1.0,C,--,--,--,2.9698,QSO,2.0,3,2.3731,0.82126,--|Strong absorption feature. RR falsely confi...,3,none


In [286]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] =  1.1797
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype, z and flag. VI z is correct. redrock thrown by fake strong absorption '

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
700,35185754023920071,-1,0.33.0.dev3890,0,QSO,2.9698,SEM,3.0,C,--,...,3003.124525,1.1797,GALAXY,2.0,3,2.3731,0.82126,--|Strong absorption feature. RR falsely confi...,3,"disagreement in spectype, z and flag. VI z is ..."
750,35185754023920071,-1,0.33.0.dev3890,0,QSO,2.9698,ASC,3.0,RCS,1.1797,...,3003.124525,1.1797,GALAXY,2.0,3,2.3731,0.82126,--|Strong absorption feature. RR falsely confi...,3,"disagreement in spectype, z and flag. VI z is ..."
800,35185754023920071,-1,0.33.0.dev3890,0,QSO,2.9698,CGS,3.0,C,--,...,3003.124525,1.1797,GALAXY,2.0,3,2.3731,0.82126,--|Strong absorption feature. RR falsely confi...,3,"disagreement in spectype, z and flag. VI z is ..."


conflict ID =  68


In [289]:
#copy this text to a new cell to display the conflict
conflict_id= 69
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
701,35185754023920128,GALAXY,0.2658,SEM,1.0,C,--,STAR,--,0.2658,STAR,0.666667,1,0.2658,0.0,--|Usual flux step error but 4700-5800AA here|--,3,none
751,35185754023920128,GALAXY,0.2658,ASC,0.0,S,--,--,Usual flux step error but 4700-5800AA here,0.2658,GALAXY,0.666667,1,0.2658,0.0,--|Usual flux step error but 4700-5800AA here|--,3,none
801,35185754023920128,GALAXY,0.2658,CGS,1.0,--,--,--,--,0.2658,GALAXY,0.666667,1,0.2658,0.0,--|Usual flux step error but 4700-5800AA here|--,3,none


In [290]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'GALAXY'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in spectype with flag 0 and 1. set to redrock.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
701,35185754023920128,-1,0.33.0.dev3890,0,GALAXY,0.2658,SEM,1.0,C,--,...,16.813944,0.2658,GALAXY,0.666667,1,0.2658,0.0,--|Usual flux step error but 4700-5800AA here|--,3,disagreement in spectype with flag 0 and 1. se...
751,35185754023920128,-1,0.33.0.dev3890,0,GALAXY,0.2658,ASC,0.0,S,--,...,16.813944,0.2658,GALAXY,0.666667,1,0.2658,0.0,--|Usual flux step error but 4700-5800AA here|--,3,disagreement in spectype with flag 0 and 1. se...
801,35185754023920128,-1,0.33.0.dev3890,0,GALAXY,0.2658,CGS,1.0,--,--,...,16.813944,0.2658,GALAXY,0.666667,1,0.2658,0.0,--|Usual flux step error but 4700-5800AA here|--,3,disagreement in spectype with flag 0 and 1. se...


conflict ID =  69


In [293]:
#copy this text to a new cell to display the conflict
conflict_id = 70
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
738,35185754023921393,GALAXY,0.4131,SEM,1.0,RC,--,--,--,0.4131,GALAXY,2.333333,3,0.4131,0.0,--|Same issue as spectrum 1|--,3,none
788,35185754023921393,GALAXY,0.4131,ASC,4.0,S,--,--,Same issue as spectrum 1,0.4131,GALAXY,2.333333,3,0.4131,0.0,--|Same issue as spectrum 1|--,3,none
838,35185754023921393,GALAXY,0.4131,CGS,2.0,--,--,--,--,0.4131,GALAXY,2.333333,3,0.4131,0.0,--|Same issue as spectrum 1|--,3,none


In [294]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag. issues in reduction, no clear features. not secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
738,35185754023921393,-1,0.33.0.dev3890,0,GALAXY,0.4131,SEM,1.0,RC,--,...,68.71637,0.4131,GALAXY,2.333333,3,0.4131,0.0,--|Same issue as spectrum 1|--,3,"disagreement in flag. issues in reduction, no ..."
788,35185754023921393,-1,0.33.0.dev3890,0,GALAXY,0.4131,ASC,1.0,S,--,...,68.71637,0.4131,GALAXY,2.333333,3,0.4131,0.0,--|Same issue as spectrum 1|--,3,"disagreement in flag. issues in reduction, no ..."
838,35185754023921393,-1,0.33.0.dev3890,0,GALAXY,0.4131,CGS,1.0,--,--,...,68.71637,0.4131,GALAXY,2.333333,3,0.4131,0.0,--|Same issue as spectrum 1|--,3,"disagreement in flag. issues in reduction, no ..."


conflict ID =  70


In [295]:
#copy this text to a new cell to display the conflict
conflict_id= 71
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
705,35185754023921559,GALAXY,0.3424,SEM,1.0,R,--,--,--,0.3424,GALAXY,1.0,2,0.3424,0.0,--|B to R flux step|--,3,none
755,35185754023921559,GALAXY,0.3424,ASC,0.0,S,--,--,B to R flux step,0.3424,GALAXY,1.0,2,0.3424,0.0,--|B to R flux step|--,3,none
805,35185754023921559,GALAXY,0.3424,CGS,2.0,--,--,--,--,0.3424,GALAXY,1.0,2,0.3424,0.0,--|B to R flux step|--,3,none


In [296]:
#copy this text to a new cell to resolve the conflict
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 1
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag 0 to 2. set to mean.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
705,35185754023921559,-1,0.33.0.dev3890,0,GALAXY,0.3424,SEM,1.0,R,--,...,7.895296,0.3424,GALAXY,1.0,2,0.3424,0.0,--|B to R flux step|--,3,disagreement in flag 0 to 2. set to mean.
755,35185754023921559,-1,0.33.0.dev3890,0,GALAXY,0.3424,ASC,1.0,S,--,...,7.895296,0.3424,GALAXY,1.0,2,0.3424,0.0,--|B to R flux step|--,3,disagreement in flag 0 to 2. set to mean.
805,35185754023921559,-1,0.33.0.dev3890,0,GALAXY,0.3424,CGS,1.0,--,--,...,7.895296,0.3424,GALAXY,1.0,2,0.3424,0.0,--|B to R flux step|--,3,disagreement in flag 0 to 2. set to mean.


conflict ID =  71


In [299]:
#copy this text to a new cell to display the conflict
conflict_id= 72
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
739,35185754023921837,GALAXY,1.6572,SEM,1.0,--,--,--,--,1.6572,GALAXY,3.0,3,0.7462,1.058728,--|Not sure what is wrong with this spectrum.|...,3,none
789,35185754023921837,GALAXY,1.6572,ASC,4.0,RS,0.2907,--,Not sure what is wrong with this spectrum.,0.2907,GALAXY,3.0,3,0.7462,1.058728,--|Not sure what is wrong with this spectrum.|...,3,none
839,35185754023921837,GALAXY,1.6572,CGS,4.0,R,0.2907,--,OII + OIII used to refit redshift plz check,0.2907,GALAXY,3.0,3,0.7462,1.058728,--|Not sure what is wrong with this spectrum.|...,3,none


In [300]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.2907
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag, but VI z is correct.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
739,35185754023921837,-1,0.33.0.dev3890,0,GALAXY,1.6572,SEM,4.0,--,--,...,187.253624,0.2907,GALAXY,3.0,3,0.7462,1.058728,--|Not sure what is wrong with this spectrum.|...,3,"disagreement in z and flag, but VI z is correct."
789,35185754023921837,-1,0.33.0.dev3890,0,GALAXY,1.6572,ASC,4.0,RS,0.2907,...,187.253624,0.2907,GALAXY,3.0,3,0.7462,1.058728,--|Not sure what is wrong with this spectrum.|...,3,"disagreement in z and flag, but VI z is correct."
839,35185754023921837,-1,0.33.0.dev3890,0,GALAXY,1.6572,CGS,4.0,R,0.2907,...,187.253624,0.2907,GALAXY,3.0,3,0.7462,1.058728,--|Not sure what is wrong with this spectrum.|...,3,"disagreement in z and flag, but VI z is correct."


conflict ID =  72


In [301]:
#copy this text to a new cell to display the conflict
conflict_id=73
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
706,35185754023922187,GALAXY,1.081,SEM,1.0,R,--,--,--,1.0810,GALAXY,1.666667,1,0.765933,0.293832,--|Possible [O II] but no corresponding [O III...,3,none
756,35185754023922187,GALAXY,1.081,ASC,2.0,RS,0.6084,GALAXY,Possible [O II] but no corresponding [O III]. ...,0.6084,GALAXY,1.666667,1,0.765933,0.293832,--|Possible [O II] but no corresponding [O III...,3,none
806,35185754023922187,GALAXY,1.081,CGS,2.0,RC,0.6084,--,OII hint,0.6084,GALAXY,1.666667,1,0.765933,0.293832,--|Possible [O II] but no corresponding [O III...,3,none


In [302]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.6084
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = '2/3 inspectors picked same z with flag 2, disagreement with 3rd. take VI z.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
706,35185754023922187,-1,0.33.0.dev3890,0,GALAXY,1.081,SEM,2.0,R,--,...,7.813437,0.6084,GALAXY,1.666667,1,0.765933,0.293832,--|Possible [O II] but no corresponding [O III...,3,"2/3 inspectors picked same z with flag 2, disa..."
756,35185754023922187,-1,0.33.0.dev3890,0,GALAXY,1.081,ASC,2.0,RS,0.6084,...,7.813437,0.6084,GALAXY,1.666667,1,0.765933,0.293832,--|Possible [O II] but no corresponding [O III...,3,"2/3 inspectors picked same z with flag 2, disa..."
806,35185754023922187,-1,0.33.0.dev3890,0,GALAXY,1.081,CGS,2.0,RC,0.6084,...,7.813437,0.6084,GALAXY,1.666667,1,0.765933,0.293832,--|Possible [O II] but no corresponding [O III...,3,"2/3 inspectors picked same z with flag 2, disa..."


conflict ID =  73


In [305]:
#copy this text to a new cell to display the conflict
conflict_id= 74
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
740,35185754023922420,GALAXY,0.4499,SEM,-1.0,RC,--,--,--,0.4499,GALAXY,0.666667,3,0.6349,0.382785,--|Same issue as spectrum 1|--,3,none
790,35185754023922420,GALAXY,0.4499,ASC,2.0,RS,1.0049,GALAXY,Same issue as spectrum 1,1.0049,GALAXY,0.666667,3,0.6349,0.382785,--|Same issue as spectrum 1|--,3,none
840,35185754023922420,GALAXY,0.4499,CGS,1.0,--,--,--,--,0.4499,GALAXY,0.666667,3,0.6349,0.382785,--|Same issue as spectrum 1|--,3,none


In [306]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0049
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 2
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z. take VI z with flag 2.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
740,35185754023922420,-1,0.33.0.dev3890,0,GALAXY,0.4499,SEM,2.0,RC,--,...,19.112736,1.0049,GALAXY,0.666667,3,0.6349,0.382785,--|Same issue as spectrum 1|--,3,disagreement in z. take VI z with flag 2.
790,35185754023922420,-1,0.33.0.dev3890,0,GALAXY,0.4499,ASC,2.0,RS,1.0049,...,19.112736,1.0049,GALAXY,0.666667,3,0.6349,0.382785,--|Same issue as spectrum 1|--,3,disagreement in z. take VI z with flag 2.
840,35185754023922420,-1,0.33.0.dev3890,0,GALAXY,0.4499,CGS,2.0,--,--,...,19.112736,1.0049,GALAXY,0.666667,3,0.6349,0.382785,--|Same issue as spectrum 1|--,3,disagreement in z. take VI z with flag 2.


conflict ID =  74


In [309]:
#copy this text to a new cell to display the conflict
conflict_id= 75
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
741,35185754023922558,GALAXY,0.505,SEM,1.0,RC,--,--,--,0.5050,GALAXY,1.666667,2,0.677233,0.343322,--|--|--,3,none
791,35185754023922558,GALAXY,0.505,ASC,3.0,RS,1.0217,--,--,1.0217,GALAXY,1.666667,2,0.677233,0.343322,--|--|--,3,none
841,35185754023922558,GALAXY,0.505,CGS,1.0,--,--,--,--,0.5050,GALAXY,1.666667,2,0.677233,0.343322,--|--|--,3,none


In [310]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0217
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag, but VI z is secure.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
741,35185754023922558,-1,0.33.0.dev3890,0,GALAXY,0.505,SEM,3.0,RC,--,...,72.155427,1.0217,GALAXY,1.666667,2,0.677233,0.343322,--|--|--,3,"disagreement in z and flag, but VI z is secure."
791,35185754023922558,-1,0.33.0.dev3890,0,GALAXY,0.505,ASC,3.0,RS,1.0217,...,72.155427,1.0217,GALAXY,1.666667,2,0.677233,0.343322,--|--|--,3,"disagreement in z and flag, but VI z is secure."
841,35185754023922558,-1,0.33.0.dev3890,0,GALAXY,0.505,CGS,3.0,--,--,...,72.155427,1.0217,GALAXY,1.666667,2,0.677233,0.343322,--|--|--,3,"disagreement in z and flag, but VI z is secure."


conflict ID =  75


In [311]:
#copy this text to a new cell to display the conflict
conflict_id=76
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
742,35185754023922751,GALAXY,0.4436,SEM,1.0,RC,--,--,--,0.4436,GALAXY,1.0,0,0.600933,0.32696,--|Same issue as spectrum 1. Unconvincing [O I...,3,none
792,35185754023922751,GALAXY,0.4436,ASC,1.0,RS,0.9156,--,Same issue as spectrum 1. Unconvincing [O II],0.9156,GALAXY,1.0,0,0.600933,0.32696,--|Same issue as spectrum 1. Unconvincing [O I...,3,none
842,35185754023922751,GALAXY,0.4436,CGS,1.0,--,--,--,--,0.4436,GALAXY,1.0,0,0.600933,0.32696,--|Same issue as spectrum 1. Unconvincing [O I...,3,none


In [312]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9156
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreemeng in z with flag 1. take VI z with flag 1'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
742,35185754023922751,-1,0.33.0.dev3890,0,GALAXY,0.4436,SEM,1.0,RC,--,...,119.015844,0.9156,GALAXY,1.0,0,0.600933,0.32696,--|Same issue as spectrum 1. Unconvincing [O I...,3,disagreemeng in z with flag 1. take VI z with ...
792,35185754023922751,-1,0.33.0.dev3890,0,GALAXY,0.4436,ASC,1.0,RS,0.9156,...,119.015844,0.9156,GALAXY,1.0,0,0.600933,0.32696,--|Same issue as spectrum 1. Unconvincing [O I...,3,disagreemeng in z with flag 1. take VI z with ...
842,35185754023922751,-1,0.33.0.dev3890,0,GALAXY,0.4436,CGS,1.0,--,--,...,119.015844,0.9156,GALAXY,1.0,0,0.600933,0.32696,--|Same issue as spectrum 1. Unconvincing [O I...,3,disagreemeng in z with flag 1. take VI z with ...


conflict ID =  76


In [315]:
#copy this text to a new cell to display the conflict
conflict_id=77
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
748,35185754023924323,GALAXY,0.5326,SEM,2.0,--,--,--,possible OIII,0.5326,GALAXY,2.666667,2,0.7078,0.342947,possible OIII|Same issue as spectrum 29|--,3,none
798,35185754023924323,GALAXY,0.5326,ASC,4.0,RS,1.0582,--,Same issue as spectrum 29,1.0582,GALAXY,2.666667,2,0.7078,0.342947,possible OIII|Same issue as spectrum 29|--,3,none
848,35185754023924323,GALAXY,0.5326,CGS,2.0,--,--,--,--,0.5326,GALAXY,2.666667,2,0.7078,0.342947,possible OIII|Same issue as spectrum 29|--,3,none


In [316]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 1.0582
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 3
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in z and flag. VI z is correct.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)



,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
748,35185754023924323,-1,0.33.0.dev3890,0,GALAXY,0.5326,SEM,3.0,--,--,...,75.293579,1.0582,GALAXY,2.666667,2,0.7078,0.342947,possible OIII|Same issue as spectrum 29|--,3,disagreement in z and flag. VI z is correct.
798,35185754023924323,-1,0.33.0.dev3890,0,GALAXY,0.5326,ASC,3.0,RS,1.0582,...,75.293579,1.0582,GALAXY,2.666667,2,0.7078,0.342947,possible OIII|Same issue as spectrum 29|--,3,disagreement in z and flag. VI z is correct.
848,35185754023924323,-1,0.33.0.dev3890,0,GALAXY,0.5326,CGS,3.0,--,--,...,75.293579,1.0582,GALAXY,2.666667,2,0.7078,0.342947,possible OIII|Same issue as spectrum 29|--,3,disagreement in z and flag. VI z is correct.


conflict ID =  77


In [319]:
#copy this text to a new cell to display the conflict
conflict_id=78
display_conflict(conflict_id, show_spectra=False)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
715,35185754028115586,QSO,1.0144,SEM,2.0,--,--,--,OII and NEIII,1.0144,QSO,2.666667,2,1.0144,0.0,OII and NEIII|Unconvincing [O II] and [Ne III...,3,disagreement in flag. set to 2.
765,35185754028115586,QSO,1.0144,ASC,2.0,S,--,--,Unconvincing [O II] and [Ne III]. Flux step.,1.0144,QSO,2.666667,2,1.0144,0.0,OII and NEIII|Unconvincing [O II] and [Ne III...,3,disagreement in flag. set to 2.
815,35185754028115586,QSO,1.0144,CGS,2.0,--,--,--,--,1.0144,QSO,2.666667,2,1.0144,0.0,OII and NEIII|Unconvincing [O II] and [Ne III...,3,disagreement in flag. set to 2.


In [322]:
#copy this text to a new cell to display the conflict
conflict_id=79
display_conflict(conflict_id, show_spectra=False)



,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
718,35185754028116041,GALAXY,0.9433,SEM,1.0,RC,1.8333,QSO,--,1.8333,QSO,2.0,3,1.5347,0.457984,--|Clear quasar. Flux large everywhere.|--,3,none
768,35185754028116041,GALAXY,0.9433,ASC,4.0,RC,1.8275,QSO,Clear quasar. Flux large everywhere.,1.8275,QSO,2.0,3,1.5347,0.457984,--|Clear quasar. Flux large everywhere.|--,3,none
818,35185754028116041,GALAXY,0.9433,CGS,1.0,--,--,--,--,0.9433,GALAXY,2.0,3,1.5347,0.457984,--|Clear quasar. Flux large everywhere.|--,3,none


In [323]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = (1.8333 + 1.8275 )/2
vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 4
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 'QSO'

#add a comment
vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 'disagreement in flag, spectype and z. clear QSO. take average of the 2 QSO VI redshifts.'

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
718,35185754028116041,-1,0.33.0.dev3890,0,GALAXY,0.9433,SEM,4.0,RC,1.8333,...,38.701416,1.8304,QSO,2.0,3,1.5347,0.457984,--|Clear quasar. Flux large everywhere.|--,3,"disagreement in flag, spectype and z. clear QS..."
768,35185754028116041,-1,0.33.0.dev3890,0,GALAXY,0.9433,ASC,4.0,RC,1.8275,...,38.701416,1.8304,QSO,2.0,3,1.5347,0.457984,--|Clear quasar. Flux large everywhere.|--,3,"disagreement in flag, spectype and z. clear QS..."
818,35185754028116041,-1,0.33.0.dev3890,0,GALAXY,0.9433,CGS,4.0,--,--,...,38.701416,1.8304,QSO,2.0,3,1.5347,0.457984,--|Clear quasar. Flux large everywhere.|--,3,"disagreement in flag, spectype and z. clear QS..."


conflict ID =  79


### and so on...

We should now recompute the conflicts, and not find any.

In [326]:
vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
                       & (len(x) >= 2)) #x is a group by TargetID

In [327]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  [35185741977881344 35185747992511204]
Total number of conflicts to resolve:  2


Display anything that was missed (if "Total number of conflicts" isn't zero) and resolve!

In [328]:
#copy this text to a new cell to display the conflict
conflict_id=0
display_conflict(conflict_id)

,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
43,35185741977881344,GALAXY,0.8796,VRK,0.666667,RS,--,--,dubious fit (no doublet + fit driven by one u...,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.
93,35185741977881344,GALAXY,0.8796,CBA,0.666667,--,--,--,--,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.
143,35185741977881344,GALAXY,0.8796,ACE,0.666667,R,1.5252,GALAXY,OII on sky line,1.5252,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.


In [329]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.8796
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)


,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
43,35185741977881344,-1,0.33.0.dev3890,0,GALAXY,0.8796,VRK,0.666667,RS,--,...,57.031874,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.
93,35185741977881344,-1,0.33.0.dev3890,0,GALAXY,0.8796,CBA,0.666667,--,--,...,57.031874,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.
143,35185741977881344,-1,0.33.0.dev3890,0,GALAXY,0.8796,ACE,0.666667,R,1.5252,...,57.031874,0.8796,GALAXY,0.666667,2,1.0948,0.343477,dubious fit (no doublet + fit driven by one u...,3,flag disagreement 0 to 2. set to mean.


conflict ID =  0


In [330]:
#copy this text to a new cell to display the conflict
conflict_id= 1
display_conflict(conflict_id, show_spectra=False)


,TargetID,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,VI spectype,VI comment,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
311,35185747992511204,GALAXY,0.9263,CBA,1.0,--,--,--,--,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."
361,35185747992511204,GALAXY,0.9263,ACE,1.0,RC,--,STAR,Much more likely to be a star,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."
411,35185747992511204,GALAXY,0.9263,vrk,1.0,RS,0.9849,--,RR doublet not convincing + VI only doublet s...,0.9849,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."


In [331]:
#copy this text to a new cell to resolve the conflict
vi.loc[vi.TargetID==unique_targets[conflict_id], 'best redshift'] = 0.9263
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'VI class'] = 
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'best spectype'] = 

#add a comment
#vi.loc[vi.TargetID==unique_targets[conflict_id], 'merger comment'] = 

#look at the values again to make sure all is well
display(vi[vi.TargetID==unique_targets[conflict_id]])
print('conflict ID = ', conflict_id)

,TargetID,ExpID,Spec version,Redrock version,Redrock spectype,Redrock z,VI scanner,VI class,VI issue,VI z,...,DELTACHI2,best redshift,best spectype,vi_combined_flag,vi_diff,vi_combined_z,dz,all VI comments,N_VI,merger comment
311,35185747992511204,-1,0.33.0.dev3890,0,GALAXY,0.9263,CBA,1.0,--,--,...,6.950417,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."
361,35185747992511204,-1,0.33.0.dev3890,0,GALAXY,0.9263,ACE,1.0,RC,--,...,6.950417,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."
411,35185747992511204,-1,0.33.0.dev3890,0,GALAXY,0.9263,vrk,1.0,RS,0.9849,...,6.950417,0.9263,GALAXY,1.333333,3,0.945833,0.030421,--| Much more likely to be a star| RR doublet ...,3,"disagreement in flag, z and spectype. very low..."


conflict ID =  1


Let's check the number of conflicts again.

In [332]:
vi_conflict = vi_gp.filter(lambda x: ( ( (x['VI class'].max()-x['VI class'].min()) >= 2) 
                       | ( (x['best redshift'].max() - x['best redshift'].min()) / (1+x['best redshift'].min()) > 0.0033 ) 
                       | (not all(i == x['best spectype'].iloc[0] for i in x['best spectype'])) )
                       & (len(x) >= 2)) #x is a group by TargetID

In [333]:
unique_targets = np.unique(vi_conflict['TargetID'].tolist())
print('Targets with problematic VI: ', unique_targets)
print('Total number of conflicts to resolve: ', len(unique_targets))

Targets with problematic VI:  []
Total number of conflicts to resolve:  0


## Woohoo!

## Finally
we need to recompute vi_combined_flag and vi_combined_z, after everything is resolved.

In [334]:
vi['vi_combined_flag'] = vi.groupby('TargetID')['VI class'].transform('mean')
vi['vi_combined_z'] = vi.groupby('TargetID')['best redshift'].transform('mean')


## Now we prepare to write to file. 

### The important columns for the truth table construction are **vi_combined_flag** and **vi_combined_z** 

The truth table should **take the redhift value in vi_combined_z** for all objects with **vi_combined_flag >= 2.5** (for discussion, but this catches the cases where there is some disagreement on the 2/3 boundary).

After the merging process, those two columns will be the same for ALL of the members of each group by TargetID, so it doesn't matter which member we write to file. Taking the 1st one just makes this easy. 


In [335]:
vi.keys()

Index(['TargetID', 'ExpID', 'Spec version', 'Redrock version',
       'Redrock spectype', 'Redrock z', 'VI scanner', 'VI class', 'VI issue',
       'VI z', 'VI spectype', 'VI comment', 'FIBER', 'FLUX_G', 'FLUX_R',
       'FLUX_Z', 'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z', 'DELTACHI2',
       'best redshift', 'best spectype', 'vi_combined_flag', 'vi_diff',
       'vi_combined_z', 'dz', 'all VI comments', 'N_VI', 'merger comment'],
      dtype='object')

In [341]:
#vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'all VI comments', 'merger comment', 'N_VI', 'DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+'merged_VI.txt')
vi_gp['Redrock z', 'Redrock spectype', 'best redshift' , 'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments', 'merger comment','DELTACHI2','FIBER','FLUX_G','FLUX_R','FLUX_Z','FIBERFLUX_G','FIBERFLUX_R','FIBERFLUX_Z'].first().to_csv(VI_dir+save_filename)

Check that merged file reads in OK - check comments

In [342]:
merged_file = pd.read_csv(VI_dir+save_filename)

In [343]:
merged_file.keys()

Index(['TargetID', 'Redrock z', 'Redrock spectype', 'best redshift',
       'best spectype', 'vi_combined_flag', 'N_VI', 'all VI comments',
       'merger comment', 'DELTACHI2', 'FIBER', 'FLUX_G', 'FLUX_R', 'FLUX_Z',
       'FIBERFLUX_G', 'FIBERFLUX_R', 'FIBERFLUX_Z'],
      dtype='object')

In [344]:
merged_file

,TargetID,Redrock z,Redrock spectype,best redshift,best spectype,vi_combined_flag,N_VI,all VI comments,merger comment,DELTACHI2,FIBER,FLUX_G,FLUX_R,FLUX_Z,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z
0,35185741969491373,0.5809,GALAXY,0.5809,GALAXY,2.000000,3,dubious fit (doublet from noise + weak Hbeta ...,"disagreement in flag and z. continuum fit, but...",6.757586,337,0.364552,0.326236,1.014764,0.148809,0.133169,0.414224
1,35185741969491540,0.9348,GALAXY,0.9348,GALAXY,3.333333,3,--| Probable [OII]|--,none,86.867207,310,0.584755,0.631218,1.512308,0.115881,0.125089,0.299694
2,35185741969491567,0.8267,GALAXY,0.8267,GALAXY,4.000000,3,"--| [OII]"","" clear abs features"","" Balmer line...",none,918.023997,330,0.799700,1.729793,5.527337,0.387193,0.837519,2.676187
3,35185741969492004,1.5427,GALAXY,1.5427,GALAXY,0.000000,3,unsecure doublet in skyline noise and not eno...,none,3.439889,309,0.294638,0.348021,0.437433,0.167634,0.198006,0.248877
4,35185741969492139,1.2281,GALAXY,1.2281,GALAXY,0.000000,3,dubious fit (negative doublet + all other lin...,none,1.849894,331,0.246348,0.156775,0.169003,0.191548,0.121900,0.131408
5,35185741969492213,1.1040,GALAXY,1.1040,GALAXY,3.333333,3,--| Probable [OII]|--,none,14.789172,315,0.296448,0.324394,0.226197,0.190310,0.208250,0.145211
6,35185741969492412,0.1617,GALAXY,0.1617,GALAXY,4.000000,3,"ELG or BGS ?| [OII]"","" Balmer em lines"","" [OI...",none,310.160914,456,0.479977,0.561181,0.651049,0.332382,0.388616,0.450849
7,35185741969492861,1.5197,GALAXY,1.5197,GALAXY,2.333333,3,broad doublet in skyline region| Possible [OII...,none,68.104874,455,0.282862,0.296696,0.780082,0.192412,0.201822,0.530637
8,35185741969492962,1.1763,GALAXY,1.1763,GALAXY,3.666667,3,--| [OII] doublet|--,none,223.460540,450,0.322885,0.366338,0.761616,0.250673,0.284408,0.591285
9,35185741969493081,0.8229,GALAXY,1.4485,GALAXY,3.000000,3,doublet mistaken as OIII line at 9130AA + VI ...,"disagreement in flag, but 3/3 inspectors agree...",47.458578,4783,0.316183,0.355301,0.688885,0.184783,0.207644,0.402597


In [345]:
merged_file.loc[9]['all VI comments']

' doublet mistaken as OIII line at 9130AA + VI redshift gives secure strong and resolved doublet| Tentative [OII] at 9130A| OII fit as OIII with double peak that is not apporpriate'

In [346]:
VI_dir

'/global/homes/r/rtojeiro/SV/VI_files/SV0/ELG/'